In [1]:
import pandas as pd

In [57]:
import csv
import random
from datetime import datetime, timedelta
import statistics

# Function to generate random transaction type based on CID
def generate_transaction_type(cid):
    if cid % 3 == 0:
        return 'Credit' if random.random() < 0.3 else 'Debit'
    else:
        return 'Debit' if random.random() > 0.3 else 'Credit'

# Generate synthetic data
def generate_synthetic_data(num_records):
    data = []
    cids = list(range(1, num_records + 1))
    num_cids_with_high_debits = round(num_records * 0.3)
    cids_with_high_debits = random.sample(cids, num_cids_with_high_debits)

    start_date = datetime(2021, 1, 1)
    end_date = datetime(2023, 12, 31)

    delta_day_brackets = [[1, 10], [15, 20], [30, 45]]  # Delta day brackets

    # Probabilities for choosing each bracket
    bracket_probabilities = [0.3, 0.3, 0.4]
    chosen_delta_days = random.choices(delta_day_brackets, weights=bracket_probabilities, k=len(cids))

    # Calculate the NEFT transaction amount for each CID based on buckets
    neft_amount_buckets = [[15000, 40000], [40000, 90000], [90000, 180000]]
    bucket_probabilities = [0.4, 0.4, 0.2]

    # Calculate the NEFT transaction amount for each CID
    neft_amounts = {}
    for i, cid in enumerate(cids):
        chosen_bucket = random.choices(neft_amount_buckets, weights=bucket_probabilities)[0]
        neft_amounts[cid] = round(random.uniform(chosen_bucket[0], chosen_bucket[1]), 2)

        balance = 10000
        num_transactions = random.randint(80, 100)
        transaction_dates = []
        previous_date = None

        total_transactions = num_records * num_transactions
        num_rtgs_transactions = round(total_transactions * 0.08)
        num_rtgs_credit = round(num_rtgs_transactions * 0.5)
        num_rtgs_debit = num_rtgs_transactions - num_rtgs_credit

        num_credit_transactions = 0  # Counter for 'Credit' transactions

        quarterly_interest_month = [3, 6, 9, 12]  # Quarterly interest months

        # Use the chosen delta day bracket for this CID
        chosen_delta_days_for_cid = chosen_delta_days[i]

        # Use the chosen delta day bracket for this CID
        chosen_delta_days_for_cid = chosen_delta_days[i]

        for _ in range(num_transactions):
            delta_days = random.randint(chosen_delta_days_for_cid[0], chosen_delta_days_for_cid[1])  # Use chosen bracket
            if previous_date is None:
                transaction_date = start_date
            else:
                transaction_date = previous_date + timedelta(days=delta_days)
                if transaction_date > end_date:
                    transaction_date = start_date
            transaction_dates.append(transaction_date)
            previous_date = transaction_date

        num_upi_debit = round(num_transactions * random.uniform(0.05, 0.3))  # 30% to 40% of 'Debit' transactions
        num_upi_debit = min(num_upi_debit, num_transactions)  # Ensure it doesn't exceed total transactions

        for i in range(num_transactions):
            transaction_date = transaction_dates[i].strftime('%Y-%m-%d')
            transaction_date_dt = datetime.strptime(transaction_date, '%Y-%m-%d')  # Convert to datetime object
            transaction_type = generate_transaction_type(cid)

            # Check for penalty condition
            if balance < 3000 and (transaction_date_dt - previous_date).days > 30:
                penalty_amount = round(random.uniform(150, 650), 2)
                balance -= penalty_amount
                data.append([cid, transaction_date, 'Debit', penalty_amount, 'Penalty', balance])

            if i == 0 or transaction_dates[i].month != transaction_dates[i - 1].month:
                neft_transaction_date = datetime(transaction_dates[i].year, transaction_dates[i].month, 1)
                neft_amount = neft_amounts[cid]  # Use the pre-calculated NEFT amount for this CID
                balance += neft_amount
                data.append([cid, neft_transaction_date.strftime('%Y-%m-%d'), 'Credit', neft_amount, 'NEFT', balance])
                num_credit_transactions += 1

            if transaction_type == 'Credit':
                narration = random.choice(['IMPS', 'UPI'])
                amount = round(random.uniform(100, 200000), 2)
                if narration == 'UPI':
                    upi_debit_amount = min(100000, amount)  # Limit UPI amount to 100,000
                    amount = round(random.uniform(10, upi_debit_amount), 2)
                num_credit_transactions += 1
                # Check if quarterly interest month and year
                if transaction_dates[i].month in quarterly_interest_month and transaction_dates[i].year >= start_date.year:
                    interest_amount = round(balance * random.uniform(0.05, 0.06), 2)  # 5% to 6% of balance
                    balance += interest_amount
                    data.append([cid, transaction_date, 'Credit', interest_amount, 'Interest', balance])
            else:
                transaction_type = 'Debit'  # Change 'Credit' to 'Debit'
                narration = random.choice(['UPI', 'RTGS', 'IMPS', 'Cash Withdrawal', 'Check Bounce', 'E-Transfer Bounce'])
                if num_rtgs_debit > 0:
                  if random.random() < 0.08 and narration == 'RTGS':  # 8% chance of RTGS for 'Debit' transactions
                    balance += 300000  # Set balance to 300000 for CIDs with 'RTGS' narration*
                    amount = round(random.uniform(200000, 300000), 2)
                    num_rtgs_debit -= 1
                  else:
                    narration = random.choice(['UPI', 'IMPS', 'Cash Withdrawal', 'Check Bounce', 'E-Transfer Bounce'])
                    max_debit_percent = 0.6
                    if cid in cids_with_high_debits:
                        max_debit_amount = balance * 0.4
                    else:
                        max_debit_amount = balance * max_debit_percent
                    amount = round(random.uniform(10, max_debit_amount), 2)
                else:
                  if narration == 'RTGS':
                    narration = random.choice(['UPI', 'IMPS', 'Cash Withdrawal', 'Check Bounce', 'E-Transfer Bounce']) 
                  max_debit_percent = 0.6
                  if cid in cids_with_high_debits:
                      max_debit_amount = balance * 0.4
                  else:
                      max_debit_amount = balance * max_debit_percent
                  amount = round(random.uniform(10, max_debit_amount), 2)

                if narration == 'UPI':
                  upi_debit_amount = min(100000, max_debit_amount)  # Limit UPI amount to 100,000
                  amount = round(random.uniform(10, upi_debit_amount), 2)
                  num_upi_debit -= 1
                  if num_upi_debit == 0:
                      narration = random.choice(['IMPS', 'Cash Withdrawal'])

                if narration == 'E-Transfer Bounce' or narration == 'Check Bounce':
                  if random.random() < 0.05:  # 5% chance of bounce narrated transactions
                    amount = random.uniform(2 * balance, 3 * balance)

            if transaction_type == 'Debit':
              if narration in ['E-Transfer Bounce', 'Check Bounce']:
                balance -= 0
              else:
                balance -= amount
            else:
                balance += amount

            data.append([cid, transaction_date, transaction_type, amount, narration, balance])

    return data

# Save synthetic data to CSV
def save_to_csv(data, filename):
    with open(filename, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(['CID', 'Transaction Date', 'Transaction Type', 'Transaction Amount', 'Narration', 'Balance'])
        csv_writer.writerows(data)

if __name__ == "__main__":
    num_records = 10000  # Number of synthetic records to generate
    synthetic_data = generate_synthetic_data(num_records)
    save_to_csv(synthetic_data, 'synthetic_transactions.csv')
    print("Synthetic data saved to synthetic_transactions.csv")

Synthetic data saved to synthetic_transactions.csv


Monthly distribution for different types
credit - 0 to 2, 2 to 8, 8 to 15 | Narrations: 'NEFT', 'UPI', 'RTGS', 'IMPS'
debit - 30 to 50, 50 to 70, 70 to 120 | Narrations: 'UPI', 'RTGS', 'Cash Withdrawal', 'Card Payment', 'IMPS', etc except 'NEFT'

Time Frame distribution:
Three months - 
Six months - 
Nine months - 
Twelve months - 


High
frequent no of transactions - (80,100)
high amount credit/debit
frequent NEFT salaries

Medium
normal no of transactions - (50, 70)
normal amount credit/debit

Low
less frequent NEFT salaries
low no of transactions - (30, 50)
low amount credit/debit
more cash withdrawals


UPI Narrations
- Grocery 
- Family
- Pharmacy/Healthcare
- Wellness and Fitness
- Autopay
- Entertainment
- Gasoline/Fuel
- Shopping
- Dining
- Ticket Booking
- Travel
- Electricity Bill
- Water Bill
- Telephone Bill
- Gas Bill
- Internet Bill
- Cable Bill
- Merchant
- Rent
- Mobile Recharge Prepaid
- Mobile Recharge Postpaid
NEFT
RTGS
IMPS
Cash Withdrawal
Loan EMI
Interest
Check Bounce
E-Transfer Bounce


In [6]:
import pandas as pd
df = pd.read_csv('D:\All_Files\Base_Data\Customer_Base_Data.csv')

In [7]:
df['Type_of_employment'].value_counts()

Type_of_employment
Un-Employed    3478
Salaried       2512
Business       2488
Pensioner      1522
Name: count, dtype: int64

### For Loyal Salaried

In [2]:
import pandas as pd
import random
from datetime import datetime, timedelta
from itertools import cycle

# Define the number of customers (CIDs) and the number of months for the dataset
num_customers = 20000

# Initialize an empty list to store the data
data = []

# Define three functions for 'high,' 'medium,' and 'low' behaviors
def high():
    return random.randint(55, 100), random.randint(2, 6)

def medium():
    return random.randint(35, 55), random.randint(2, 4)

def low():
    return random.randint(20, 35), random.randint(1, 2)

def narration(transaction_type, transaction_date, balance, debit_narrations, bacs_months, interest_months, Autopay_months,Rent_months,Loan_emi_months,Loan_emi,cid,selected_CID_Bounces,count_bounces,chaps_threshold,chaps_count):
    transaction_date = datetime.strptime(transaction_date, '%d-%m-%Y')
    current_month = (transaction_date.year, transaction_date.month)

    # Handling 'Credit' transactions
    if transaction_type == 'Credit':
        if current_month not in bacs_months:
            narration = 'BACS'  # Salary credited once a month
            bacs_months.add(current_month)  # Record the year and month as a tuple
        elif current_month not in interest_months:
            narration = 'Interest'  # Interest credited once a month
            interest_months.add(current_month)  # Record the year and month as a tuple
        else:
            if random.random() < 0.05:
                narration = random.choices(['CHAPS', 'Faster Payment'], weights = [0.05,0.95],k=1)[0]
            else:
                narration = 'Faster Payment'
    # Handling 'Debit' transactions
    else:
        valid_narration = False
        if current_month not in Autopay_months and Autopay < balance:
            narration = 'Autopay'  # Salary credited once a month
            Autopay_months.add(current_month)  # Record the year and month as a tuple
            valid_narration = True
        elif current_month not in Rent_months and Rent < balance:
            narration = 'Rent'  # Interest credited once a month
            Rent_months.add(current_month)
            valid_narration = True
        elif current_month not in Loan_emi_months and Loan_emi < balance and Loan_emi != 0:
            narration = 'Loan EMI'  # Interest credited once a month
            Loan_emi_months.add(current_month)
            valid_narration = True
        elif cid in selected_CID_Bounces and count_bounces <=2:
            narration = random.choice(['Check Bounce','E-Transfer Bounce'])
            valid_narration = True   
        while not valid_narration:
            # Assign weights to different narrations
            weights = {'CHAPS': 0.01, 'Cash Withdrawal': 0.10,  
                        'Grocery':0.2, "Pharmacy/Healthcare":0.1, "Wellness and Fitness":0.05, "Entertainment":0.08,
                        "Gasoline/Fuel": 0.1, "Shopping": 0.15, "Dining": 0.10, "Ticket Booking": 0.03, "Travel": 0.05, "Merchant": 0.02,
                        "Mobile Recharge": 0.01, "Electricity Bill": 0.01, "Water Bill":0.01, "Telephone Bill":0.01, "Gas Bill":0.01, "Internet Bill":0.01, "Cable Bill":0.01}  # Assigning a default weight for custom narrations
            
            # Remove keys from 'weights' that match the values in 'debit_narrations'
            updated_weights = {key: value for key, value in weights.items() if key in debit_narrations}
            
            # Randomly select a narration based on weights
            narration = random.choices(list(updated_weights.keys()), weights=list(updated_weights.values()), k=1)[0]       
            
            # Check if selected narration is 'CHAPS' and meets balance threshold
            if (narration != 'CHAPS' or (narration == 'CHAPS' and balance > 10000 and chaps_count<=chaps_threshold)):
                if narration in selected_utility.keys() and selected_utility[narration] == 0:
                    selected_utility[narration]+=1
                    valid_narration = True
                elif narration not in selected_utility.keys():
                    valid_narration = True
    return narration

# Function to calculate the transaction amount
def amount(balance, transaction_type, transaction_function,salary, current_behavior_index, narration, Autopay, Rent,Loan_emi,narration_amount):
    if transaction_type == 'Credit' and narration == 'BACS':
        return salary
    elif transaction_type == 'Credit' and narration == 'Interest':
        return round(balance * 0.05,2)
    elif transaction_type == 'Credit' and narration == 'CHAPS':
        # Ensure the amount is more than 10000 and less than 60% of the balance
        lower_bound = salary * 0.8   # Lower bound is the greater of 10000 or 10% of balance
        upper_bound = max(5000, salary * 1.2)  # Upper bound is 60% of the balance
        return round(random.uniform(lower_bound, upper_bound), 2)
    elif transaction_type == 'Credit':
        return round(salary * (random.uniform(0.01, 0.1)), 2)
    else:
    # For Debit transactions
        if transaction_type == 'Debit' and narration == 'CHAPS':
            return round(balance * (random.uniform(0.1, 0.4)), 2)
        if transaction_type == 'Debit' and (narration == 'Check Bounce' or narration == 'E-Transfer Bounce'):
            return 0
        if transaction_type == 'Debit' and narration == 'Autopay':
            return Autopay
        if transaction_type == 'Debit' and narration == 'Rent':
            return Rent
        if transaction_type == 'Debit' and narration == 'Loan EMI':
            return Loan_emi
        if transaction_type == 'Debit' and narration in narration_amount:
            value = round(random.uniform(5, narration_amount[narration]), 2)
            if value < balance:
                return value
            else:
                if transaction_function[current_behavior_index] in [low, medium]:
                        if random.random() < 0.9:  # 90% chance to use a higher percentage
                            debit_percentage = random.uniform(0.05, 0.15)
                        else:
                            debit_percentage = random.uniform(0.01, 0.05)
                else:
                    debit_percentage = random.uniform(0.01, 0.1)

                if round(salary * debit_percentage, 2) > balance:
                    return round(balance * debit_percentage, 2)
                else:
                    return round(salary * debit_percentage, 2)            

def modify_narration(amount_value,current_narration):
    same = current_narration
    if current_narration in ["Grocery", "Pharmacy/Healthcare", "Wellness and Fitness", "Entertainment",
                             "Gasoline/Fuel", "Shopping", "Dining", "Ticket Booking", "Travel", "Merchant",
                             "Mobile Recharge", "Electricity Bill", "Water Bill", "Telephone Bill", "Gas Bill",
                             "Internet Bill", "Cable Bill"] and amount_value<=30:
        prefix = random.choices(['Fast Payment_', 'Cash_'], weights=[0.6, 0.4])[0]
        current_narration = prefix + current_narration
    elif current_narration in ["Grocery", "Pharmacy/Healthcare", "Wellness and Fitness", "Entertainment",
                             "Gasoline/Fuel", "Shopping", "Dining", "Ticket Booking", "Travel", "Merchant",
                             "Mobile Recharge", "Electricity Bill", "Water Bill", "Telephone Bill", "Gas Bill",
                             "Internet Bill", "Cable Bill"] and amount_value>30:
        prefix = random.choices(['Fast Payment_', 'Debit Card_', 'Cash_'], weights=[0.3, 0.4, 0.3])[0]
        current_narration = prefix + current_narration
    elif current_narration in ['Loan EMI', 'Rent']:
        prefix = 'Standing Order_'
        current_narration = prefix + current_narration
    elif current_narration == 'Autopay':
        prefix = 'Direct Debit_'
        current_narration = prefix + current_narration
    else:
        current_narration = same

    return current_narration

# Create a list of functions to randomly select from
transaction_functions = [high, medium, low]

# Define the number of customers for each behavior group
num_low_customers = 5000
num_medium_customers = 4000
num_high_customers = 4000
num_high_customers_2 = 4000
num_random_customers = num_customers - (num_low_customers + num_medium_customers + num_high_customers + num_high_customers_2)

# Create lists of customer behaviors for each group
low_behaviors = [low] * 5 + [medium] * 5 + [high] * (25 - 5 - 5)
medium_behaviors = [medium] * 8 + [high] * (25 - 8)
high_behaviors_1 = [high] * 3 + [medium] * 4 + [low] * 5 + [high] * (25 - 3 - 4 - 5)
high_behaviors_2 = [high] * 3 + [medium] * 4 + [high] * (25 - 3 - 4)
a5 = [random.choice(transaction_functions) for _ in range(26)]

# Generate a list of numbers from 1 to 10000
all_numbers = list(range(1, 20000))
# Calculate the number of elements to select (half of the list)
num_of_CID_having_loans = len(all_numbers) // 2
# Randomly select half of the numbers
selected_CID = random.sample(all_numbers, num_of_CID_having_loans)

num_of_CID_having_Bounces = int(0.05*len(all_numbers))
selected_CID_Bounces = random.sample(all_numbers, num_of_CID_having_Bounces)

# Generate synthetic data for each customer
for cid in range(1, num_customers + 1):
    balance = round(random.uniform(1000,2500),2)
    current_date = datetime(2021, 1, 1)
    a = 0
    # Select a range for the salary
    salary_range = random.choices([[2500, 5000], [5000, 7000], [7000, 9000]], weights=[0.25, 0.45, 0.30], k=1)[0]
    # Now use random.uniform to select a value within the chosen range
    salary = random.uniform(salary_range[0], salary_range[1])
    
    categories = ["CHAPS","Grocery","Cash Withdrawal", "Pharmacy/Healthcare", "Wellness and Fitness", "Entertainment",
                        "Gasoline/Fuel", "Shopping", "Dining", "Ticket Booking", "Travel", "Merchant",
                        "Mobile Recharge", "Electricity Bill", "Water Bill", "Telephone Bill", "Gas Bill", "Internet Bill", "Cable Bill"]

    random.shuffle(categories)
    
    num_items_to_choose = int(1 * len(categories))
    debit_narrations = categories[:num_items_to_choose]
    bacs_months = set()
    interest_months = set()
    Autopay_months = set()
    Rent_months = set()
    Autopay = round(random.uniform(20,60),2)
    Rent = round(random.uniform(salary*0.1,salary*0.3),2)
    if cid in selected_CID:
        Loan_emi = round(random.uniform(salary*0.1,salary*0.5),2)
        Loan_emi_months = set()
    else:
        Loan_emi = 0
        Loan_emi_months = set()
    count_bounces = 0
    
    ### CHAPS Control
    # Define the list with corresponding weightages
    numbers = [[0, 1], [2, 5], [6, 25]]
    weights = [0.1, 0.4, 0.5]
    # Select a list based on the weights
    selected_list = random.choices(numbers, weights=weights, k=1)[0]
    # Get the minimum and maximum values from the selected list
    min_val, max_val = min(selected_list), max(selected_list)
    # Generate a random integer within the range of the selected sublist
    chaps_threshold = random.randint(min_val, max_val)
    chaps_count = 0    
    
    while current_date <= datetime(2023, 1, 31):
        if cid <=5000:
            transaction_function = low_behaviors
        elif cid <= 4000:
            transaction_function = medium_behaviors
        elif cid <= 4000:
            transaction_function = high_behaviors_1
        elif cid <= 4000:
            transaction_function = high_behaviors_2
        else:
            transaction_function = a5

        num_transactions, num_credit = transaction_function[a]()
        num_debit = num_transactions - num_credit
        a += 1
        
        selected_utility = {"Electricity Bill":0, "Water Bill":0, "Telephone Bill":0, "Gas Bill":0, "Internet Bill":0, "Cable Bill":0}
        narration_amount = {
            'Grocery': round(random.uniform(15, 200), 2),
            'Pharmacy/Healthcare': round(random.uniform(10, 400), 2),
            'Wellness and Fitness': round(random.uniform(40, 500), 2),
            'Entertainment': round(random.uniform(40, 350), 2),
            'Gasoline/Fuel': round(random.uniform(20, 220), 2),
            'Shopping': round(random.uniform(40, 600), 2),
            'Dining': round(random.uniform(20, 250), 2),
            'Ticket Booking': round(random.uniform(25, 145), 2),
            'Travel': round(random.uniform(50, 550), 2),
            'Merchant': round(random.uniform(25, 290), 2),
            'Mobile Recharge': round(random.uniform(5, 80), 2),
            'Cash Withdrawal': round(random.uniform(50, 500), 2),
            'Electricity Bill': round(random.uniform(5, 80), 2),
            'Water Bill': round(random.uniform(5, 60), 2),
            'Telephone Bill': round(random.uniform(5, 80), 2),
            'Gas Bill': round(random.uniform(5, 75), 2),
            'Internet Bill': round(random.uniform(10, 60), 2),
            'Cable Bill': round(random.uniform(10, 65), 2)
            }

        last_day_of_month = (current_date.replace(day=1) + timedelta(days=32)).replace(day=1) - timedelta(days=1)
        transaction_dates = [current_date + timedelta(days=random.randint(0, (last_day_of_month - current_date).days)) for _ in range(num_transactions)]
        transaction_dates.sort()
        date_iterator = cycle(transaction_dates)
        
        for _ in range(num_transactions):
            transaction_date = next(date_iterator).strftime('%d-%m-%Y')
            if num_credit > 0 and num_debit > 0:
                transaction_type = random.choice(['Credit', 'Debit'])
            elif num_credit > 0:
                transaction_type = 'Credit'
            else:
                transaction_type = 'Debit'
                
            if transaction_type == 'Credit':
                num_credit-=1
            else:
                num_debit-=1

            current_narration = narration(transaction_type,transaction_date, balance,debit_narrations,bacs_months, interest_months,Autopay_months,Rent_months,Loan_emi_months,Loan_emi,cid,selected_CID_Bounces,count_bounces,chaps_threshold,chaps_count)
            if current_narration == 'Check Bounce' or 'E-Transfer Bounce':
                count_bounces+=1
            elif current_narration == 'CHAPS':
                chaps_count+=1
            
            amount_value = round(amount(balance, transaction_type, transaction_function,salary,a-1, current_narration, Autopay, Rent,Loan_emi,narration_amount),2)

            balance = round(balance - amount_value if transaction_type == 'Debit' else balance + amount_value, 2)
            
            current_narration = modify_narration(amount_value,current_narration)
            
            transaction_date = datetime.strptime(transaction_date, '%d-%m-%Y')
            current_month = (transaction_date.year, transaction_date.month)
            
            # Assume this is inside a loop
            exit_loop = False  # Flag variable to control loop termination

            if (balance > 20 and amount_value > 1) or (current_narration in ['Check Bounce', 'E-Transfer Bounce']):
                data.append([cid, transaction_date, transaction_type, amount_value, current_narration, balance])
            elif balance < 20 and amount_value < 1:
                if current_narration in ['Check Bounce', 'E-Transfer Bounce']:
                    data.append([cid, transaction_date, transaction_type, amount_value, current_narration, balance])
                elif current_month not in interest_months:
                    if transaction_type == 'Debit':
                        balance += (2 * amount_value)
                        data.append([cid, transaction_date, 'Credit', amount_value, 'Interest', balance])
                    else:
                        data.append([cid, transaction_date, transaction_type, amount_value, 'Interest', balance])
                    exit_loop = True  # Set the flag to exit the loop

            if exit_loop:
                break  # Break out of the loop if the flag is set
            
        current_date = last_day_of_month + timedelta(days=1)

# Create a DataFrame from the generated data
df = pd.DataFrame(data, columns=['CID', 'Transaction Date', 'Transaction Type', 'Amount', 'Narration', 'Balance'])

In [3]:
df['Balance'].mean()

9758.751740075242

In [4]:
df

,CID,Transaction Date,Transaction Type,Amount,Narration,Balance
0,1,2021-01-03,Debit,32.12,Direct Debit_Autopay,2168.72
1,1,2021-01-05,Debit,1316.20,Standing Order_Rent,852.52
2,1,2021-01-06,Credit,7268.82,BACS,8121.34
3,1,2021-01-06,Debit,1407.64,Standing Order_Loan EMI,6713.70
4,1,2021-01-07,Debit,145.93,Cash_Grocery,6567.77
...,...,...,...,...,...,...
20811595,20000,2023-01-29,Debit,12.87,Fast Payment_Pharmacy/Healthcare,1523.04
20811596,20000,2023-01-29,Debit,22.44,Fast Payment_Entertainment,1500.60
20811597,20000,2023-01-29,Debit,11.50,Cash_Grocery,1489.10
20811598,20000,2023-01-30,Debit,94.21,Cash_Shopping,1394.89


In [5]:
df.to_csv('Salaried_Loyal_Part5.csv', index =False)

In [1]:
df

NameError: name 'df' is not defined

In [32]:
pd.set_option('display.max_rows', 2000)  # Change 50 to your desired number of rows

In [28]:
df[(df['CID'] == 7) & (df['Balance']<1000)]

,CID,Transaction Date,Transaction Type,Amount,Narration,Balance


In [21]:
df.to_csv('Loyal_Salaried-Part-3.csv', index =False)

### For At-Risk Salaried

In [56]:
import pandas as pd
import random
from datetime import datetime, timedelta
from itertools import cycle

# Define the number of customers (CIDs) and the number of months for the dataset
num_customers = 18750

# Initialize an empty list to store the data
data = []

# Define three functions for 'high,' 'medium,' and 'low' behaviors
def high():
    return random.randint(40, 50), random.randint(2, 4)

def medium():
    return random.randint(30, 40), random.randint(1, 3)

def low():
    return random.randint(20, 30), random.randint(1, 2)

def narration(transaction_type, transaction_date, balance, debit_narrations, bacs_months, interest_months, Autopay_months,Rent_months,Loan_emi_months,Loan_emi,cid, cid_to_stop_BACS, cid_for_CHAPS, cut_off_month,starting_month,chaps_count,chaps_threshold):
    transaction_date = datetime.strptime(transaction_date, '%d-%m-%Y')
    current_month = (transaction_date.year, transaction_date.month)
    
    check = 0
    
    # Handling 'Credit' transactions
    if transaction_type == 'Credit':
        if (current_month not in bacs_months): 
            if (cid in cid_to_stop_BACS and current_month <= cut_off_month) or (cid not in cid_to_stop_BACS):
                narration = 'BACS'  # Salary credited once a month
                check+=1
            bacs_months.add(current_month)  # Record the year and month as a tuple
        if current_month not in interest_months and check == 0:
            narration = 'Interest'  # Interest credited once a month
            interest_months.add(current_month)  # Record the year and month as a tuple
        elif check == 0:
            narration = 'Faster Payment'

    # Handling 'Debit' transactions
    else:
        valid_narration = False
        if cid in cid_for_CHAPS and current_month > starting_month:
            if chaps_count > chaps_threshold:
                narration = 'BACS'
            else:
                narration = 'CHAPS'
            valid_narration = True
        elif current_month not in Autopay_months and Autopay < balance and ((cid in cid_to_stop_BACS and current_month <= cut_off_month)  or (cid in cid_with_high_debit)):
            narration = 'Autopay'  # Salary credited once a month
            Autopay_months.add(current_month)  # Record the year and month as a tuple
            valid_narration = True
        elif current_month not in Rent_months and Rent < balance and ((cid in cid_to_stop_BACS and current_month <= cut_off_month)  or (cid in cid_with_high_debit)):
            narration = 'Rent'  # Interest credited once a month
            Rent_months.add(current_month)
            valid_narration = True
        elif current_month not in Loan_emi_months and Loan_emi < balance and Loan_emi != 0 and ((cid in cid_to_stop_BACS and current_month <= cut_off_month)  or (cid in cid_with_high_debit)):
            narration = 'Loan EMI'  # Interest credited once a month
            Loan_emi_months.add(current_month)
            valid_narration = True                     
        while not valid_narration:
            # Assign weights to different narrations
            weights = {'CHAPS': 0.35, 'Cash Withdrawal': 0.40, 'Check Bounce': 0.05, 'E-Transfer Bounce': 0.05 ,
                        'Grocery':0.05, "Pharmacy/Healthcare":0.05, "Wellness and Fitness":0.05, "Entertainment":0.05,
                        "Gasoline/Fuel": 0.05, "Shopping": 0.05, "Dining": 0.05, "Ticket Booking": 0.03, "Travel": 0.05, "Merchant": 0.02,
                        "Mobile Recharge": 0.01, "Electricity Bill": 0.01, "Water Bill":0.01, "Telephone Bill":0.01, "Gas Bill":0.01, "Internet Bill":0.01, "Cable Bill":0.01}  # Assigning a default weight for custom narrations
            
            # Remove keys from 'weights' that match the values in 'debit_narrations'
            updated_weights = {key: value for key, value in weights.items() if key in debit_narrations}
            
            # Randomly select a narration based on weights
            narration = random.choices(list(updated_weights.keys()), weights=list(updated_weights.values()), k=1)[0]       
            
            # Check if selected narration is 'CHAPS' and meets balance threshold
            if (narration != 'CHAPS' or (narration == 'CHAPS' and balance > 2000)):
                if narration == 'CHAPS' and chaps_count > chaps_threshold:
                    narration = "BACS"
                if narration in selected_utility.keys() and selected_utility[narration] == 0:
                    selected_utility[narration]+=1
                    valid_narration = True
                elif narration not in selected_utility.keys():
                    valid_narration = True
    return narration

# Function to calculate the transaction amount
def amount(balance, transaction_type, transaction_function,salary, current_behavior_index, narration, Autopay, Rent,Loan_emi,narration_amount):
    if transaction_type == 'Credit' and narration == 'BACS':
        return salary
    elif transaction_type == 'Credit' and narration == 'Interest':
        return round(balance * 0.05,2)
    elif transaction_type == 'Credit':
        return round(salary * (random.uniform(0.01, 0.03)), 2)
    else:
     # For Debit transactions
        if transaction_type == 'Debit' and (narration == 'CHAPS' or narration == 'BACS'):
            return round(balance * (random.uniform(0.7, 0.9)), 2)
        if transaction_type == 'Debit' and (narration == 'Check Bounce' or narration == 'E-Transfer Bounce'):
            return 0
        if transaction_type == 'Debit' and narration == 'Autopay':
            return Autopay
        if transaction_type == 'Debit' and narration == 'Rent':
            return Rent
        if transaction_type == 'Debit' and narration == 'Loan EMI':
            return Loan_emi
        if transaction_type == 'Debit' and narration in narration_amount:
            value = round(random.uniform(5, narration_amount[narration]), 2)
            if value < balance:
                return value
            else:
                if transaction_function[current_behavior_index] in [low, medium]:
                        if random.random() < 0.9:  # 90% chance to use a higher percentage
                            debit_percentage = random.uniform(0.05, 0.15)
                        else:
                            debit_percentage = random.uniform(0.01, 0.05)
                else:
                    debit_percentage = random.uniform(0.01, 0.1)

                if round(salary * debit_percentage, 2) > balance:
                    return round(balance * debit_percentage, 2)
                else:
                    return round(salary * debit_percentage, 2)
            
def modify_narration(amount_value,current_narration):
    same = current_narration
    if current_narration in ["Grocery", "Pharmacy/Healthcare", "Wellness and Fitness", "Entertainment",
                             "Gasoline/Fuel", "Shopping", "Dining", "Ticket Booking", "Travel", "Merchant",
                             "Mobile Recharge", "Electricity Bill", "Water Bill", "Telephone Bill", "Gas Bill",
                             "Internet Bill", "Cable Bill"] and amount_value<=30:
        prefix = random.choices(['Fast Payment_', 'Cash_'], weights=[0.6, 0.4])[0]
        current_narration = prefix + current_narration
    elif current_narration in ["Grocery", "Pharmacy/Healthcare", "Wellness and Fitness", "Entertainment",
                             "Gasoline/Fuel", "Shopping", "Dining", "Ticket Booking", "Travel", "Merchant",
                             "Mobile Recharge", "Electricity Bill", "Water Bill", "Telephone Bill", "Gas Bill",
                             "Internet Bill", "Cable Bill"] and amount_value>30:
        prefix = random.choices(['Fast Payment_', 'Debit Card_', 'Cash_'], weights=[0.3, 0.4, 0.3])[0]
        current_narration = prefix + current_narration
    elif current_narration in ['Loan EMI', 'Rent']:
        prefix = 'Standing Order_'
        current_narration = prefix + current_narration
    elif current_narration == 'Autopay':
        prefix = 'Direct Debit_'
        current_narration = prefix + current_narration
    else:
        current_narration = same

    return current_narration

# Create a list of functions to randomly select from
transaction_functions = [high, medium, low]

# Define the number of customers for each behavior group
num_low_customers = 7500
num_medium_customers = 2875
num_high_customers = 3000
num_high_customers_2 = 3000
num_random_customers = num_customers - (num_low_customers + num_medium_customers + num_high_customers + num_high_customers_2)

# Create lists of customer behaviors for each group
low_behaviors = [high] * 5 + [medium] * 5 + [low] * (25 - 5 - 5)
medium_behaviors = [medium] * 8 + [low] * (25 - 8)
high_behaviors_1 = [low] * 3 + [medium] * 4 + [high] * 5 + [low] * (25 - 3 - 4 - 5)
high_behaviors_2 = [low] * 3 + [medium] * 4 + [low] * (25 - 3 - 4)
a5 = [random.choice(transaction_functions) for _ in range(26)]

# Generate a list of numbers from 1 to 10000
all_numbers = list(range(1, 18751))
# Calculate the number of elements to select (half of the list)
num_of_CID_having_loans = len(all_numbers) // 2
# Randomly select half of the numbers
selected_CID = random.sample(all_numbers, num_of_CID_having_loans)

num_of_CID_having_Bounces = int(0.3*len(all_numbers))
selected_CID_Bounces = random.sample(all_numbers, num_of_CID_having_Bounces)

# Define the range of numbers
start_range = 1
end_range = 18750

# Calculate 30%, 50%, and 20% of the range
total_numbers = end_range - start_range + 1
thirty_percent = int(0.3 * total_numbers)
fifty_percent = int(0.5 * total_numbers)
twenty_percent = total_numbers - (thirty_percent + fifty_percent)

# Generate a list of all numbers in the range
all_numbers = list(range(start_range, end_range + 1))
cid_to_stop_BACS = random.sample(all_numbers, thirty_percent)
remaining_after_30 = [num for num in all_numbers if num not in cid_to_stop_BACS]
cid_for_CHAPS = random.sample(all_numbers, fifty_percent)
cid_with_high_debit = [num for num in remaining_after_30 if num not in cid_for_CHAPS]

# Generate synthetic data for each customer
for cid in range(1, num_customers + 1):
    balance = round(random.uniform(1000,2500),2)
    current_date = datetime(2021, 1, 1)
    a = 0    
    
    if cid in cid_to_stop_BACS:
        months = [(2021, 7), (2021, 8), (2021, 9), (2021, 10),(2021, 11), (2021, 12), (2022, 1), (2022, 2), (2022, 3)]
        # Randomly select a tuple from the list
        cut_off_month = random.choice(months)
    else:
        cut_off_month = (2023,2)
    if cid in cid_for_CHAPS:
        months_tuple = ((2021, 4), (2021, 5), (2021, 6), (2021, 7),(2021, 8), (2021, 9), (2021, 10), (2021, 11), (2021, 12))
        # Randomly select a tuple from the provided tuples
        starting_month = random.choice(months_tuple)
    else:
        starting_month = (2023,2)
    
    # Select a range for the salary
    salary_range = random.choices([[2500, 5000], [5000, 7000], [7000, 9000]], weights=[0.25, 0.45, 0.30], k=1)[0]
    # Now use random.uniform to select a value within the chosen range
    salary = random.uniform(salary_range[0], salary_range[1])
    
    categories = ["Grocery","Cash Withdrawal", "Pharmacy/Healthcare", "Wellness and Fitness", "Entertainment",
                        "Gasoline/Fuel", "Shopping", "Dining", "Ticket Booking", "Travel", "Merchant",
                        "Mobile Recharge", "Electricity Bill", "Water Bill", "Telephone Bill", "Gas Bill", "Internet Bill", "Cable Bill"]

    random.shuffle(categories)    
    
    num_items_to_choose = int(0.5 * len(categories))
    debit_narrations = categories[:num_items_to_choose]
    debit_narrations.extend(["CHAPS",'Check Bounce','E-Transfer Bounce'])
    bacs_months = set()
    interest_months = set()
    Autopay_months = set()
    Rent_months = set()
    Autopay = round(random.uniform(20,60),2)
    Rent = round(random.uniform(salary*0.1,salary*0.3),2)
    if cid in selected_CID:
        Loan_emi = round(random.uniform(salary*0.1,salary*0.3),2)
        Loan_emi_months = set()
    else:
        Loan_emi = 0
        Loan_emi_months = set() 
        
    ### CHAPS Control
    # Define the list with corresponding weightages
    numbers = [[0, 20], [20, 40], [40, 65]]
    weights = [0.1, 0.4, 0.5]
    # Select a list based on the weights
    selected_list = random.choices(numbers, weights=weights, k=1)[0]
    # Get the minimum and maximum values from the selected list
    min_val, max_val = min(selected_list), max(selected_list)
    # Generate a random integer within the range of the selected sublist
    chaps_threshold = random.randint(min_val, max_val)
    chaps_count = 0   
    
    while current_date <= datetime(2023, 1, 31):
        if cid <=7500:
            transaction_function = low_behaviors
        elif cid <= 10375:
            transaction_function = medium_behaviors
        elif cid <= 13375:
            transaction_function = high_behaviors_1
        elif cid <= 16375:
            transaction_function = high_behaviors_2
        else:
            transaction_function = a5

        num_transactions, num_credit = transaction_function[a]()
        num_debit = num_transactions - num_credit
        a += 1
        
        selected_utility = {"Electricity Bill":0, "Water Bill":0, "Telephone Bill":0, "Gas Bill":0, "Internet Bill":0, "Cable Bill":0}
        narration_amount = {
            'Grocery': round(random.uniform(15, 200), 2),
            'Pharmacy/Healthcare': round(random.uniform(10, 400), 2),
            'Wellness and Fitness': round(random.uniform(40, 500), 2),
            'Entertainment': round(random.uniform(40, 350), 2),
            'Gasoline/Fuel': round(random.uniform(20, 220), 2),
            'Shopping': round(random.uniform(40, 600), 2),
            'Dining': round(random.uniform(20, 250), 2),
            'Ticket Booking': round(random.uniform(25, 145), 2),
            'Travel': round(random.uniform(50, 550), 2),
            'Merchant': round(random.uniform(25, 290), 2),
            'Mobile Recharge': round(random.uniform(5, 80), 2),
            'Cash Withdrawal': round(random.uniform(50, 500), 2),
            'Electricity Bill': round(random.uniform(5, 80), 2),
            'Water Bill': round(random.uniform(5, 60), 2),
            'Telephone Bill': round(random.uniform(5, 80), 2),
            'Gas Bill': round(random.uniform(5, 75), 2),
            'Internet Bill': round(random.uniform(10, 60), 2),
            'Cable Bill': round(random.uniform(10, 65), 2)
            }

        last_day_of_month = (current_date.replace(day=1) + timedelta(days=32)).replace(day=1) - timedelta(days=1)
        transaction_dates = [current_date + timedelta(days=random.randint(0, (last_day_of_month - current_date).days)) for _ in range(num_transactions)]
        transaction_dates.sort()
        date_iterator = cycle(transaction_dates)
        
        for _ in range(num_transactions):
            transaction_date = next(date_iterator).strftime('%d-%m-%Y')
            if num_credit > 0 and num_debit > 0:
                transaction_type = random.choice(['Credit', 'Debit'])
            elif num_credit > 0:
                transaction_type = 'Credit'
            else:
                transaction_type = 'Debit'
                
            if transaction_type == 'Credit':
                num_credit-=1
            else:
                num_debit-=1

            current_narration = narration(transaction_type,transaction_date, balance,debit_narrations,bacs_months, interest_months,Autopay_months,Rent_months,Loan_emi_months,Loan_emi,cid,cid_to_stop_BACS,cid_for_CHAPS, cut_off_month,starting_month,chaps_count,chaps_threshold)
            if current_narration == 'CHAPS' and transaction_type == 'Debit':
                chaps_count+=1
            
            amount_value = round(amount(balance, transaction_type, transaction_function,salary,a-1, current_narration, Autopay, Rent,Loan_emi,narration_amount),2)

            balance = round(balance - amount_value if transaction_type == 'Debit' else balance + amount_value, 2)
            
            current_narration = modify_narration(amount_value,current_narration)
            
            transaction_date = datetime.strptime(transaction_date, '%d-%m-%Y')
            current_month = (transaction_date.year, transaction_date.month)
            
            # Assume this is inside a loop
            exit_loop = False  # Flag variable to control loop termination

            if (balance > 20 and amount_value > 1) or (current_narration in ['Check Bounce', 'E-Transfer Bounce']):
                data.append([cid, transaction_date, transaction_type, amount_value, current_narration, balance])
            elif balance < 20 and amount_value < 1:
                if current_narration in ['Check Bounce', 'E-Transfer Bounce']:
                    data.append([cid, transaction_date, transaction_type, amount_value, current_narration, balance])
                elif current_month not in interest_months:
                    if transaction_type == 'Debit':
                        balance += (2 * amount_value)
                        data.append([cid, transaction_date, 'Credit', amount_value, 'Interest', balance])
                    else:
                        data.append([cid, transaction_date, transaction_type, amount_value, 'Interest', balance])
                    exit_loop = True  # Set the flag to exit the loop

            if exit_loop:
                break  # Break out of the loop if the flag is set
            
        current_date = last_day_of_month + timedelta(days=1)

# Create a DataFrame from the generated data
df = pd.DataFrame(data, columns=['CID', 'Transaction Date', 'Transaction Type', 'Amount', 'Narration', 'Balance'])

In [58]:
df['Balance'].mean()

1468.008068109334

## Adding Time

In [26]:
df = pd.read_csv(r"D:\All_Files\Savings_Account\2.0\Augment\3.0\Salaried_Loyal_Part4.csv")

In [27]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

# Define time ranges and their corresponding percentages
time_ranges = [
    (6, 12),    # 06:00 - 12:00
    (12, 18),   # 12:00 - 18:00
    (18, 21),   # 18:00 - 21:00
    (21, 24),   # 21:00 - 00:00
    (0, 3),     # 00:00 - 03:00
    (3, 6)      # 03:00 - 06:00
]
percentages = [0.25, 0.30, 0.20, 0.15, 0.05, 0.05]

# Number of transactions
num_transactions = len(df)

# Generate random transaction times
transaction_times = []

for time_range, percentage in zip(time_ranges, percentages):
    num_times = int(num_transactions * percentage)
    for _ in range(num_times):
        start_hour, end_hour = time_range
        # Generate random minute and second
        minute = random.randint(0, 59)
        second = random.randint(0, 59)
        # Generate random hour within the specified range
        if start_hour <= end_hour:
            hour = random.randint(start_hour, end_hour - 1)
        else:
            hour = random.randint(start_hour, 23) if random.choice([True, False]) else random.randint(0, end_hour - 1)
        # Create time string
        time_str = f"{hour:02}:{minute:02}:{second:02}"
        transaction_times.append(time_str)

# If there are not enough times due to rounding, fill the rest with random times
while len(transaction_times) < num_transactions:
    time_str = f"{random.randint(0, 23):02}:{random.randint(0, 59):02}:{random.randint(0, 59):02}"
    transaction_times.append(time_str)

# If there are too many times due to rounding, truncate the list
transaction_times = transaction_times[:num_transactions]

# Shuffle the times to ensure randomness
random.shuffle(transaction_times)

# Add 'Transaction Time' column to DataFrame
df['Transaction Time'] = transaction_times

In [28]:
df.to_csv('Salaried_Loyal_Part4.csv', index =False)

## For Medium Salaried

In [86]:
import pandas as pd
import random
from datetime import datetime, timedelta
from itertools import cycle

# Define the number of customers (CIDs) and the number of months for the dataset
num_customers = 35000

# Initialize an empty list to store the data
data = []

# Define three functions for 'high,' 'medium,' and 'low' behaviors
def high():
    return random.randint(35, 60), random.randint(5, 12)

def medium():
    return random.randint(15, 35), random.randint(4, 8)

def low():
    return random.randint(5, 15), random.randint(2, 4)

def narration(transaction_type, transaction_date, balance, debit_narrations, bacs_months, interest_months, Autopay_months,Rent_months,Loan_emi_months,Loan_emi,cid, cid_to_stop_BACS, cid_for_CHAPS, cut_off_month,starting_month,chaps_threshold,chaps_count):
    transaction_date = datetime.strptime(transaction_date, '%d-%m-%Y')
    current_month = (transaction_date.year, transaction_date.month)

    check = 0

# Handling 'Credit' transactions
    if transaction_type == 'Credit':
        if (current_month not in bacs_months): 
            if (cid in cid_to_stop_BACS and current_month <= cut_off_month) or (cid not in cid_to_stop_BACS):
                narration = 'BACS'  # Salary credited once a month
                check+=1
            bacs_months.add(current_month)  # Record the year and month as a tuple
        if current_month not in interest_months and check == 0:
            narration = 'Interest'  # Interest credited once a month
            interest_months.add(current_month)  # Record the year and month as a tuple
        elif check == 0:
            narration = 'Faster Payment'

# Handling 'Debit' transactions
    else:
        valid_narration = False
        if cid in cid_for_CHAPS and current_month > starting_month:
            if chaps_count > chaps_threshold:
                narration = 'BACS'
            else:
                narration = 'CHAPS'
            valid_narration = True
        elif current_month not in Autopay_months and Autopay < balance and ((cid in cid_to_stop_BACS and current_month <= cut_off_month)  or (cid in cid_with_high_debit)):
            narration = 'Autopay'  # Salary credited once a month
            Autopay_months.add(current_month)  # Record the year and month as a tuple
            valid_narration = True
        elif current_month not in Rent_months and Rent < balance and ((cid in cid_to_stop_BACS and current_month <= cut_off_month)  or (cid in cid_with_high_debit)):
            narration = 'Rent'  # Interest credited once a month
            Rent_months.add(current_month)
            valid_narration = True
        elif current_month not in Loan_emi_months and Loan_emi < balance and Loan_emi != 0 and ((cid in cid_to_stop_BACS and current_month <= cut_off_month)  or (cid in cid_with_high_debit)):
            narration = 'Loan EMI'  # Interest credited once a month
            Loan_emi_months.add(current_month)
            valid_narration = True                     
        while not valid_narration:
            # Assign weights to different narrations
            weights = {'CHAPS': 0.03, 'Cash Withdrawal': 0.10, 'Check Bounce': 0.03, 'E-Transfer Bounce': 0.03 ,
                        'Grocery':0.06, "Pharmacy/Healthcare":0.04, "Wellness and Fitness":0.04, "Entertainment":0.04,
                        "Gasoline/Fuel": 0.05, "Shopping": 0.04, "Dining": 0.04, "Ticket Booking": 0.04, "Travel": 0.04, "Merchant": 0.02,
                        "Mobile Recharge": 0.01, "Electricity Bill": 0.01, "Water Bill":0.01, "Telephone Bill":0.01, "Gas Bill":0.01, "Internet Bill":0.01, "Cable Bill":0.01}  # Assigning a default weight for custom narrations
            
            # Remove keys from 'weights' that match the values in 'debit_narrations'
            updated_weights = {key: value for key, value in weights.items() if key in debit_narrations}
            
            # Randomly select a narration based on weights
            narration = random.choices(list(updated_weights.keys()), weights=list(updated_weights.values()), k=1)[0]       
            
            # Check if selected narration is 'CHAPS' and meets balance threshold
            if (narration != 'CHAPS' or (narration == 'CHAPS' and balance > 2000)):
                if narration == 'CHAPS' and chaps_count > chaps_threshold:
                    narration = 'BACS'
                if narration in selected_utility.keys() and selected_utility[narration] == 0:
                    selected_utility[narration]+=1
                    valid_narration = True
                elif narration not in selected_utility.keys():
                    valid_narration = True
    return narration

# Function to calculate the transaction amount
def amount(balance, transaction_type, transaction_function,salary, current_behavior_index, narration, Autopay, Rent,Loan_emi,narration_amount):
    if transaction_type == 'Credit' and narration == 'BACS':
        return salary
    elif transaction_type == 'Credit' and narration == 'Interest':
        return round(balance * 0.05,2)
    elif transaction_type == 'Credit':
        return round(salary * (random.uniform(0.01, 0.13)), 2)
    else:
    # For Debit transactions
        if transaction_type == 'Debit' and (narration == 'CHAPS' or narration == 'BACS'):
            return round(balance * (random.uniform(0.1, 0.4)), 2)
        if transaction_type == 'Debit' and (narration == 'Check Bounce' or narration == 'E-Transfer Bounce'):
            return 0
        if transaction_type == 'Debit' and narration == 'Autopay':
            return Autopay
        if transaction_type == 'Debit' and narration == 'Rent':
            return Rent
        if transaction_type == 'Debit' and narration == 'Loan EMI':
            return Loan_emi
        if transaction_type == 'Debit' and narration in narration_amount:
            value = round(random.uniform(5, narration_amount[narration]), 2)
            if value < balance:
                return value
            else:
                if transaction_function[current_behavior_index] in [low, medium]:
                        if random.random() < 0.9:  # 90% chance to use a higher percentage
                            debit_percentage = random.uniform(0.05, 0.15)
                        else:
                            debit_percentage = random.uniform(0.01, 0.05)
                else:
                    debit_percentage = random.uniform(0.01, 0.1)

                if round(salary * debit_percentage, 2) > balance:
                    return round(balance * debit_percentage, 2)
                else:
                    return round(salary * debit_percentage, 2)
            

def modify_narration(amount_value,current_narration):
    same = current_narration
    if current_narration in ["Grocery", "Pharmacy/Healthcare", "Wellness and Fitness", "Entertainment",
                             "Gasoline/Fuel", "Shopping", "Dining", "Ticket Booking", "Travel", "Merchant",
                             "Mobile Recharge", "Electricity Bill", "Water Bill", "Telephone Bill", "Gas Bill",
                             "Internet Bill", "Cable Bill"] and amount_value<=30:
        prefix = random.choices(['Fast Payment_', 'Cash_'], weights=[0.6, 0.4])[0]
        current_narration = prefix + current_narration
    elif current_narration in ["Grocery", "Pharmacy/Healthcare", "Wellness and Fitness", "Entertainment",
                             "Gasoline/Fuel", "Shopping", "Dining", "Ticket Booking", "Travel", "Merchant",
                             "Mobile Recharge", "Electricity Bill", "Water Bill", "Telephone Bill", "Gas Bill",
                             "Internet Bill", "Cable Bill"] and amount_value>30:
        prefix = random.choices(['Fast Payment_', 'Debit Card_', 'Cash_'], weights=[0.3, 0.4, 0.3])[0]
        current_narration = prefix + current_narration
    elif current_narration in ['Loan EMI', 'Rent']:
        prefix = 'Standing Order_'
        current_narration = prefix + current_narration
    elif current_narration == 'Autopay':
        prefix = 'Direct Debit_'
        current_narration = prefix + current_narration
    else:
        current_narration = same

    return current_narration

# Create a list of functions to randomly select from
transaction_functions = [high, medium, low]

# Define the number of customers for each behavior group
num_low_customers = 16000
num_medium_customers = 7000
num_high_customers = 4500
num_high_customers_2 = 4500
num_random_customers = num_customers - (num_low_customers + num_medium_customers + num_high_customers + num_high_customers_2)

# Create lists of customer behaviors for each group
low_behaviors = [high] * 5 + [medium] * 15 + [low] * (25 - 5 - 15)
medium_behaviors = [medium] * 18 + [low] * (25 - 18)
high_behaviors_1 = [low] * 3 + [medium] * 4 + [high] * 5 + [medium] * (25 - 3 - 4 - 5)
high_behaviors_2 = [low] * 3 + [medium] * 14 + [high] * (25 - 3 - 14)
a5 = [random.choice(transaction_functions) for _ in range(26)]

# Generate a list of numbers from 1 to 10000
all_numbers = list(range(1, 35001))
# Calculate the number of elements to select (half of the list)
num_of_CID_having_loans = len(all_numbers) // 2
# Randomly select half of the numbers
selected_CID = random.sample(all_numbers, num_of_CID_having_loans)

num_of_CID_having_Bounces = int(0.3*len(all_numbers))
selected_CID_Bounces = random.sample(all_numbers, num_of_CID_having_Bounces)

# Define the range of numbers
start_range = 1
end_range = 35000

# Calculate 30%, 50%, and 20% of the range
total_numbers = end_range - start_range + 1
thirty_percent = int(0.2 * total_numbers)
fifty_percent = int(0.3 * total_numbers)
twenty_percent = total_numbers - (thirty_percent + fifty_percent)

# Generate a list of all numbers in the range
all_numbers = list(range(start_range, end_range + 1))
cid_to_stop_BACS = random.sample(all_numbers, thirty_percent)
remaining_after_30 = [num for num in all_numbers if num not in cid_to_stop_BACS]
cid_for_CHAPS = random.sample(remaining_after_30, fifty_percent)
cid_with_high_debit = [num for num in remaining_after_30 if num not in cid_for_CHAPS]

# Generate synthetic data for each customer
for cid in range(1, num_customers + 1):
    balance = round(random.uniform(1000,5000),2)
    current_date = datetime(2021, 1, 1)
    a = 0    
    
    if cid in cid_to_stop_BACS:
        months = [(2022, 7), (2022, 8),(2022, 9),(2022, 10), (2022, 11), (2022, 12),(2023, 1)]
        # Randomly select a tuple from the list
        cut_off_month = random.choice(months)
    else:
        cut_off_month = (2023,2)
    if cid in cid_for_CHAPS:
        months_tuple = ((2022, 7), (2022, 8), (2022, 9), (2022, 10),(2022, 11), (2022, 12), (2023, 1))
        # Randomly select a tuple from the provided tuples
        starting_month = random.choice(months_tuple)
    else:
        starting_month = (2023,2)

    # Select a range for the salary
    salary_range = random.choices([[1500, 5000], [5000, 10000], [10000, 15000]], weights=[0.25, 0.45, 0.30], k=1)[0]
    # Now use random.uniform to select a value within the chosen range
    salary = random.uniform(salary_range[0], salary_range[1])

    categories = ["CHAPS","Grocery","Cash Withdrawal","Pharmacy/Healthcare", "Wellness and Fitness", "Entertainment",
                        "Gasoline/Fuel", "Shopping", "Dining", "Ticket Booking", "Travel", "Merchant",
                        "Mobile Recharge", "Electricity Bill", "Water Bill", "Telephone Bill", "Gas Bill", "Internet Bill", "Cable Bill"]

    random.shuffle(categories)    
    
    num_items_to_choose = int(0.75 * len(categories))
    debit_narrations = categories[:num_items_to_choose]
    debit_narrations.extend(['Check Bounce','E-Transfer Bounce'])
    bacs_months = set()
    interest_months = set()
    Autopay_months = set()
    Rent_months = set()
    Autopay = round(random.uniform(20,60),2)
    Rent = round(random.uniform(salary*0.1,salary*0.3),2)
    if cid in selected_CID:
        Loan_emi = round(random.uniform(salary*0.02,salary*0.08),2)
        Loan_emi_months = set()
    else:
        Loan_emi = 0
        Loan_emi_months = set()
    
    ### CHAPS Control
    # Define the list with corresponding weightages
    numbers = [[0, 15], [15, 32], [32, 50]]
    weights = [0.1, 0.4, 0.5]
    
    # Select a list based on the weights
    selected_list = random.choices(numbers, weights=weights, k=1)[0]
    # Get the minimum and maximum values from the selected list
    min_val, max_val = min(selected_list), max(selected_list)
    # Generate a random integer within the range of the selected sublist
    chaps_threshold = random.randint(min_val, max_val)
    chaps_count = 0   
    
    while current_date <= datetime(2023, 1, 31):
        if cid <= 16000:
            transaction_function = low_behaviors
        elif cid <= 23000:
            transaction_function = medium_behaviors
        elif cid <= 27500:
            transaction_function = high_behaviors_1
        elif cid <= 32000:
            transaction_function = high_behaviors_2
        else:
            transaction_function = a5

        num_transactions, num_credit = transaction_function[a]()
        num_debit = num_transactions - num_credit
        a += 1
        
        selected_utility = {"Electricity Bill":0, "Water Bill":0, "Telephone Bill":0, "Gas Bill":0, "Internet Bill":0, "Cable Bill":0}
        narration_amount = {
            'Grocery': round(random.uniform(15, 200), 2),
            'Pharmacy/Healthcare': round(random.uniform(10, 400), 2),
            'Wellness and Fitness': round(random.uniform(40, 500), 2),
            'Entertainment': round(random.uniform(40, 350), 2),
            'Gasoline/Fuel': round(random.uniform(20, 220), 2),
            'Shopping': round(random.uniform(40, 600), 2),
            'Dining': round(random.uniform(20, 250), 2),
            'Ticket Booking': round(random.uniform(25, 145), 2),
            'Travel': round(random.uniform(50, 550), 2),
            'Merchant': round(random.uniform(25, 290), 2),
            'Mobile Recharge': round(random.uniform(5, 80), 2),
            'Cash Withdrawal': round(random.uniform(50, 500), 2),
            'Electricity Bill': round(random.uniform(5, 80), 2),
            'Water Bill': round(random.uniform(5, 60), 2),
            'Telephone Bill': round(random.uniform(5, 80), 2),
            'Gas Bill': round(random.uniform(5, 75), 2),
            'Internet Bill': round(random.uniform(10, 60), 2),
            'Cable Bill': round(random.uniform(10, 65), 2)
            }

        last_day_of_month = (current_date.replace(day=1) + timedelta(days=32)).replace(day=1) - timedelta(days=1)
        transaction_dates = [current_date + timedelta(days=random.randint(0, (last_day_of_month - current_date).days)) for _ in range(num_transactions)]
        transaction_dates.sort()
        date_iterator = cycle(transaction_dates)
        
        for _ in range(num_transactions):
            transaction_date = next(date_iterator).strftime('%d-%m-%Y')
            if num_credit > 0 and num_debit > 0:
                transaction_type = random.choice(['Credit', 'Debit'])
            elif num_credit > 0:
                transaction_type = 'Credit'
            else:
                transaction_type = 'Debit'
                
            if transaction_type == 'Credit':
                num_credit-=1
            else:
                num_debit-=1

            current_narration = narration(transaction_type,transaction_date, balance,debit_narrations,bacs_months, interest_months,Autopay_months,Rent_months,Loan_emi_months,Loan_emi,cid,cid_to_stop_BACS,cid_for_CHAPS, cut_off_month,starting_month,chaps_threshold,chaps_count)
            if current_narration == 'CHAPS':
                chaps_count+=1            
            amount_value = round(amount(balance, transaction_type, transaction_function,salary,a-1, current_narration, Autopay, Rent,Loan_emi,narration_amount),2)

            balance = round(balance - amount_value if transaction_type == 'Debit' else balance + amount_value, 2)

            current_narration = modify_narration(amount_value,current_narration)
            
            transaction_date = datetime.strptime(transaction_date, '%d-%m-%Y')
            current_month = (transaction_date.year, transaction_date.month)
            
            # Assume this is inside a loop
            exit_loop = False  # Flag variable to control loop termination

            if (balance > 20 and amount_value > 1) or (current_narration in ['Check Bounce', 'E-Transfer Bounce']):
                data.append([cid, transaction_date, transaction_type, amount_value, current_narration, balance])
            elif balance < 20 and amount_value < 1:
                if current_narration in ['Check Bounce', 'E-Transfer Bounce']:
                    data.append([cid, transaction_date, transaction_type, amount_value, current_narration, balance])
                elif current_month not in interest_months:
                    if transaction_type == 'Debit':
                        balance += (2 * amount_value)
                        data.append([cid, transaction_date, 'Credit', amount_value, 'Interest', balance])
                    else:
                        data.append([cid, transaction_date, transaction_type, amount_value, 'Interest', balance])
                    exit_loop = True  # Set the flag to exit the loop

            if exit_loop:
                break  # Break out of the loop if the flag is set
        current_date = last_day_of_month + timedelta(days=1)

# Create a DataFrame from the generated data
df = pd.DataFrame(data, columns=['CID', 'Transaction Date', 'Transaction Type', 'Amount', 'Narration', 'Balance'])

KeyboardInterrupt: 

In [84]:
len(selected_CID_Bounces)

500

In [31]:
df2 = df[df['CID']<=10]

In [34]:
df2.to_csv('Salaried_10CID.csv', index =False)

In [131]:
df['Narration'].value_counts()

Narration
BACS                                 877767
Cash Withdrawal                      749662
Faster Payment                       650499
CHAPS                                483276
Interest                             440663
E-Transfer Bounce                    396500
Check Bounce                         395249
Direct Debit_Autopay                 191562
Standing Order_Rent                  187806
Standing Order_Loan EMI               91720
Fast Payment_Grocery                  66235
Fast Payment_Dining                   63767
Fast Payment_Gasoline/Fuel            63417
Fast Payment_Pharmacy/Healthcare      61634
Fast Payment_Entertainment            58673
Fast Payment_Wellness and Fitness     58325
Fast Payment_Shopping                 57361
Fast Payment_Travel                   57122
Cash_Grocery                          52987
Cash_Dining                           52135
Cash_Gasoline/Fuel                    51679
Cash_Pharmacy/Healthcare              51608
Cash_Entertainment    

In [99]:
df[(df['Narration'] == 'BACS')]

,CID,Transaction Date,Transaction Type,Amount,Narration,Balance
0,1,2021-01-01,Credit,5371.84,BACS,10228.18
38,1,2021-02-02,Credit,5371.84,BACS,12156.01
86,1,2021-03-01,Credit,5371.84,BACS,13073.37
140,1,2021-04-04,Credit,5371.84,BACS,16447.33
184,1,2021-05-01,Credit,5371.84,BACS,19284.74
...,...,...,...,...,...,...
7648830,18750,2022-09-03,Credit,1576.95,BACS,15004.48
7648870,18750,2022-10-03,Credit,1576.95,BACS,15161.97
7648897,18750,2022-11-02,Credit,1576.95,BACS,14963.42
7648956,18750,2022-12-07,Credit,1576.95,BACS,14100.75


In [202]:
df

,CID,Transaction Date,Transaction Type,Amount,Narration,Balance
0,1,2021-01-03,Credit,2565.95,BACS,4498.60
1,1,2021-01-04,Credit,224.93,Interest,4723.53
2,1,2021-01-04,Debit,47.48,Direct Debit_Autopay,4676.05
3,1,2021-01-05,Debit,271.62,Standing Order_Rent,4404.43
4,1,2021-01-05,Debit,201.41,Standing Order_Loan EMI,4203.02
...,...,...,...,...,...,...
5548654,10000,2023-01-27,Debit,20.92,Fast Payment_Ticket Booking,1715.77
5548655,10000,2023-01-28,Debit,50.54,Fast Payment_Wellness and Fitness,1665.23
5548656,10000,2023-01-30,Debit,82.84,Fast Payment_Gasoline/Fuel,1582.39
5548657,10000,2023-01-31,Debit,37.18,Cash_Gasoline/Fuel,1545.21


In [132]:
df.to_csv('At-Risk Salaried V2.csv', index = False)

In [51]:
df1= pd.read_csv('D:\All_Files\Savings_Account\\2.0\Loyal_Salaried.csv')

In [ ]:
df1

In [92]:
df[df['Narration'] == 'Interest']

,CID,Transaction Date,Transaction Type,Amount,Narration,Balance
2,1,2021-01-02,Credit,507.59,Interest,10659.37
40,1,2021-02-03,Credit,592.35,Interest,12439.25
88,1,2021-03-01,Credit,653.67,Interest,13727.04
141,1,2021-04-04,Credit,822.37,Interest,17269.70
185,1,2021-05-01,Credit,964.24,Interest,20248.98
...,...,...,...,...,...,...
7648836,18750,2022-09-06,Credit,721.81,Interest,15158.01
7648872,18750,2022-10-04,Credit,734.72,Interest,15429.10
7648899,18750,2022-11-03,Credit,748.17,Interest,15711.59
7648957,18750,2022-12-07,Credit,705.04,Interest,14805.79


In [5]:
df[df['Narration'] == 'Check Bounce']

,CID,Transaction Date,Transaction Type,Amount,Narration,Balance
14,1,2021-01-09,Debit,0.0,Check Bounce,10077.66
15,1,2021-01-11,Debit,0.0,Check Bounce,10077.66
20,1,2021-01-13,Debit,0.0,Check Bounce,8717.64
49,1,2021-02-02,Debit,0.0,Check Bounce,8971.78
70,1,2021-02-18,Debit,0.0,Check Bounce,9983.81
...,...,...,...,...,...,...
22458399,35000,2022-09-30,Debit,0.0,Check Bounce,61406.28
22458409,35000,2022-10-13,Debit,0.0,Check Bounce,77819.40
22458430,35000,2022-11-06,Debit,0.0,Check Bounce,95782.01
22458436,35000,2022-11-12,Debit,0.0,Check Bounce,98910.49


In [110]:
cid_to_stop_BACS

[9360,
 1261,
 6891,
 7011,
 6710,
 4850,
 7992,
 7541,
 278,
 7608,
 5781,
 473,
 730,
 1394,
 5739,
 5008,
 142,
 2221,
 8369,
 2672,
 8435,
 1530,
 7714,
 2786,
 8883,
 6155,
 4457,
 4625,
 4315,
 913,
 4232,
 1561,
 18,
 7788,
 6129,
 4243,
 6006,
 1639,
 6018,
 2628,
 5063,
 7499,
 1600,
 2831,
 8844,
 2307,
 3422,
 6772,
 9634,
 8020,
 9666,
 9327,
 7370,
 2437,
 684,
 3055,
 4760,
 2301,
 5700,
 9405,
 7147,
 9457,
 6315,
 4877,
 7487,
 8388,
 3739,
 6952,
 7385,
 7519,
 4939,
 2605,
 9627,
 3335,
 4101,
 4277,
 1590,
 8523,
 8232,
 42,
 2331,
 1087,
 9945,
 8138,
 1060,
 1436,
 6671,
 3397,
 436,
 4086,
 8529,
 6237,
 7896,
 9864,
 1182,
 4711,
 8059,
 7660,
 5143,
 3511,
 1745,
 796,
 5217,
 4303,
 4738,
 7641,
 9915,
 6682,
 2745,
 8571,
 46,
 8975,
 7796,
 4121,
 2016,
 1532,
 2472,
 9434,
 721,
 9226,
 7701,
 5526,
 2183,
 1476,
 6669,
 1470,
 9430,
 698,
 1965,
 679,
 1973,
 8194,
 6706,
 2698,
 8228,
 7428,
 9462,
 8323,
 7077,
 8348,
 3857,
 3432,
 4433,
 953,
 2686,
 59

In [12]:
df3 = df[(df['CID'] == 34988)]

In [13]:
df3

,CID,Transaction Date,Transaction Type,Amount,Narration,Balance
22450001,34988,2021-01-01,Credit,5660.47,BACS,9003.88
22450002,34988,2021-01-01,Credit,450.19,Interest,9454.07
22450003,34988,2021-01-02,Credit,401.75,Faster Payment,9855.82
22450004,34988,2021-01-03,Credit,202.65,Faster Payment,10058.47
22450005,34988,2021-01-03,Debit,2361.89,CHAPS,7696.58
...,...,...,...,...,...,...
22450644,34988,2023-01-20,Debit,408.69,CHAPS,1515.41
22450645,34988,2023-01-20,Debit,385.43,CHAPS,1129.98
22450646,34988,2023-01-25,Debit,276.48,CHAPS,853.50
22450647,34988,2023-01-27,Credit,42.68,Interest,896.18


In [162]:
df3.to_csv('Loyal_Business_Sample.csv', index = False)

In [110]:
df[(df['Narration']=='BACS') & (df["Transaction Type"] == 'Debit')]

,CID,Transaction Date,Transaction Type,Amount,Narration,Balance
659,2,2021-07-11,Debit,78.67,BACS,22.52
663,2,2021-08-04,Debit,6107.00,BACS,2310.31
664,2,2021-08-05,Debit,1952.93,BACS,357.38
665,2,2021-08-05,Debit,251.43,BACS,105.95
666,2,2021-08-07,Debit,78.23,BACS,27.72
...,...,...,...,...,...,...
6208571,18750,2022-09-16,Debit,1214.89,BACS,277.12
6208572,18750,2022-09-17,Debit,212.92,BACS,64.20
6208577,18750,2022-10-16,Debit,4755.49,BACS,1420.14
6208578,18750,2022-10-25,Debit,1087.74,BACS,332.40


In [120]:
df2 = df[df['Amount'] < 1]
df2['Narration'].value_counts()

Narration
Check Bounce         505516
E-Transfer Bounce    503809
Interest              87047
Name: count, dtype: int64

In [ ]:
df3

In [133]:
df2 = df[df['Balance'] < 100]
df2['CID'].nunique()

18497

In [ ]:
df[df['CID']]

In [70]:
df2['CID'].nunique()

13789

In [154]:
len(df3)

927

In [113]:
df.to_csv('At-Risk_Salaried.csv', index = False)

In [59]:
import pandas as pd
from datetime import datetime

df['Transaction Date'] = pd.to_datetime(df['Transaction Date'], format='%Y-%m-%d')

# Define the date ranges
date_ranges = [
    ('2021-01-01', '2021-03-31'),
    ('2021-04-01', '2021-06-30'),
    ('2021-07-01', '2021-09-30'),
    ('2021-10-01', '2021-12-31'),
    ('2022-01-01', '2022-03-31'),
    ('2022-04-01', '2022-06-30'),
    ('2022-07-01', '2022-09-30'),
    ('2022-10-01', '2022-12-31')
]

# Function to convert date format
def convert_date_format(date_str, current_format='%Y-%m-%d', new_format='%d-%m-%Y'):
    return datetime.strptime(date_str, current_format).strftime(new_format)

# Convert the dates in the ranges to the new format
converted_date_ranges = [(convert_date_format(start), convert_date_format(end)) for start, end in date_ranges]

# Print the converted date ranges
date_ranges = converted_date_ranges

# Function to calculate rolling average for a given date range
def rolling_avg_for_range(df, cid_col, balance_col, start_date, end_date):
    mask = (df['Transaction Date'] >= start_date) & (df['Transaction Date'] <= end_date)
    filtered_df = df[mask]
    rolling_avg = filtered_df.groupby(cid_col)[balance_col].mean()
    return rolling_avg

# Create a new DataFrame to store results
new_df = pd.DataFrame({'CID': df['CID'].unique()})

# Calculate and store rolling averages
for i, (start, end) in enumerate(date_ranges):
    col_name = f'rolling_avg_{start}_to_{end}'
    rolling_avg = rolling_avg_for_range(df, 'CID', 'Balance', start, end)
    new_df = new_df.merge(rolling_avg, left_on='CID', right_index=True, how='left', suffixes=('', f'_{col_name}'))

# Define function to count occurrences of a condition
def count_occurrences(df, cid_col, condition_col, condition):
    occurrences = df[df[condition_col] < condition].groupby(cid_col).size().reset_index(name=f'Count_{condition_col}')
    return occurrences.set_index('CID')  # Set the index to CID

# Calculate Count_minimum_balance
count_min_balance = count_occurrences(df, 'CID', 'Balance', 100)
new_df = new_df.merge(count_min_balance, left_on='CID', right_index=True, how='left')
new_df = new_df.rename(columns={'Count_Balance': 'Count_minimum_balance_Total'})

# Calculate Count_transfer_bounce including 'Cheque Bounce' and 'E-Transfer Bounce'
bounce_conditions = ['Cheque Bounce', 'E-Transfer Bounce']

# Create a Series to count transfers with bounce conditions
count_transfer_bounce = df[df['Narration'].isin(bounce_conditions)].groupby('CID').size()
count_transfer_bounce.name = 'Count_transfer_bounce'  # Assign a name to the Series
# Merge and add the named Series to new_df
new_df = new_df.merge(count_transfer_bounce, left_on='CID', right_index=True, how='left')
new_df = new_df.rename(columns={0: 'Count_transfer_bounce'})

# Assuming df and new_df are already defined

# Create a filter for rows where 'Narration' is 'CHAPS' and 'Transaction Type' is 'Debit'
condition = (df['Narration'].str.contains('CHAPS')) & (df['Transaction Type'] == 'Debit')

# Create a Series to count transfers with the specified conditions
count_transfer_bounce = df[condition].groupby('CID').size()
count_transfer_bounce.name = 'Count_CHAPS_Debit'  # Assign a name to the Series

# Merge the Series into new_df
new_df = new_df.merge(count_transfer_bounce, left_on='CID', right_index=True, how='left')
new_df = new_df.rename(columns={0: 'Count_CHAPS_Debit'})


def count_occurrences(df, cid_col, condition_col, condition, start_date, end_date, date_format):
    # Convert 'start_date' and 'end_date' to the required date format
    start_date = pd.to_datetime(start_date, format=date_format)
    end_date = pd.to_datetime(end_date, format=date_format)
    # Convert the 'Transaction Date' column to the required date format
    df['Transaction Date'] = pd.to_datetime(df['Transaction Date'], format=date_format)
    # Apply date filtering conditions
    mask = (df['Transaction Date'] >= start_date) & (df['Transaction Date'] <= end_date)
    # Apply condition filtering
    mask &= df[condition_col] < condition
    # Group by 'CID' and count occurrences
    occurrences = df[mask].groupby(cid_col).size().reset_index(name=f'Count_{condition_col}')   
    # Set the index to 'CID'
    return occurrences.set_index('CID')

# Specify the date format (e.g., 'YYYY-MM-DD')
date_format = '%Y-%m-%d'

# Calculate Count_min_balance_dip_F6M (from 2022-01-01 to 2022-06-30)
Count_min_balance_dip_F6M = count_occurrences(
    df, 'CID', 'Balance', 100, '2021-01-01', '2021-06-30', date_format)
new_df = new_df.merge(Count_min_balance_dip_F6M, left_on='CID', right_index=True, how='left')
new_df = new_df.rename(columns={'Count_Balance': 'Count_min_balance_dip_F6M'})

# Calculate Count_min_balance_dip_F6M (from 2022-01-01 to 2022-06-30)
Count_min_balance_dip_F6M = count_occurrences(
    df, 'CID', 'Balance', 100, '2021-07-01', '2021-12-31', date_format)
new_df = new_df.merge(Count_min_balance_dip_F6M, left_on='CID', right_index=True, how='left')
new_df = new_df.rename(columns={'Count_Balance': 'Count_min_balance_dip_S6M'})

# Calculate Count_min_balance_dip_F6M (from 2022-01-01 to 2022-06-30)
Count_min_balance_dip_F6M = count_occurrences(
    df, 'CID', 'Balance', 100, '2022-01-01', '2022-06-30', date_format)
new_df = new_df.merge(Count_min_balance_dip_F6M, left_on='CID', right_index=True, how='left')
new_df = new_df.rename(columns={'Count_Balance': 'Count_min_balance_dip_T6M'})

# Calculate Count_min_balance_dip_L6M (from 2022-07-01 to 2022-12-31)
Count_min_balance_dip_L6M = count_occurrences(
    df, 'CID', 'Balance', 100, '2022-07-01', '2022-12-31', date_format)
new_df = new_df.merge(Count_min_balance_dip_F6M, left_on='CID', right_index=True, how='left')
new_df = new_df.rename(columns={'Count_Balance': 'Count_min_balance_dip_L6M'})

# Define function to count transactions based on transaction type and time period
def count_transactions(df, cid_col, trans_type_col, start_date, end_date,date_format):
    # Convert 'start_date' and 'end_date' to the required date format
    start_date = pd.to_datetime(start_date, format=date_format)
    end_date = pd.to_datetime(end_date, format=date_format)

    # Convert the 'Transaction Date' column to the required date format
    df['Transaction Date'] = pd.to_datetime(df['Transaction Date'], format=date_format)

    mask = (df['Transaction Date'] >= start_date) & (df['Transaction Date'] <= end_date)
    filtered_df = df[mask]
    transactions_count = filtered_df.groupby([cid_col, trans_type_col]).size().unstack(fill_value=0)
    return transactions_count

# Specify the date format (e.g., 'YYYY-MM-DD')
date_format = '%Y-%m-%d'

# Calculate Count_debit_trans_list_F6M per CID
debit_counts_F6M = count_transactions(df, 'CID', 'Transaction Type', '2021-01-01', '2021-06-30',date_format)
new_df = new_df.merge(debit_counts_F6M[['Debit']], left_on='CID', right_index=True, how='left')
new_df = new_df.rename(columns={'Debit': 'Count_debit_trans_list_F6M'})

debit_counts_F6M = count_transactions(df, 'CID', 'Transaction Type', '2021-07-01', '2021-12-31',date_format)
new_df = new_df.merge(debit_counts_F6M[['Debit']], left_on='CID', right_index=True, how='left')
new_df = new_df.rename(columns={'Debit': 'Count_debit_trans_list_S6M'})

debit_counts_F6M = count_transactions(df, 'CID', 'Transaction Type', '2022-01-01', '2022-06-30',date_format)
new_df = new_df.merge(debit_counts_F6M[['Debit']], left_on='CID', right_index=True, how='left')
new_df = new_df.rename(columns={'Debit': 'Count_debit_trans_list_T6M'})

# Calculate Count_debit_trans_list_L6M per CID
debit_counts_L6M = count_transactions(df, 'CID', 'Transaction Type', '2022-07-01', '2022-12-31',date_format)
new_df = new_df.merge(debit_counts_L6M[['Debit']], left_on='CID', right_index=True, how='left')
new_df = new_df.rename(columns={'Debit': 'Count_debit_trans_list_L6M'})

# Calculate Count_credit_trans_list_F6M per CID
credit_counts_F6M = count_transactions(df, 'CID', 'Transaction Type', '2021-01-01', '2021-06-30',date_format)
new_df = new_df.merge(credit_counts_F6M[['Credit']], left_on='CID', right_index=True, how='left')
new_df = new_df.rename(columns={'Credit': 'Count_credit_trans_list_F6M'})

# Calculate Count_credit_trans_list_F6M per CID
credit_counts_F6M = count_transactions(df, 'CID', 'Transaction Type', '2021-07-01', '2021-12-31',date_format)
new_df = new_df.merge(credit_counts_F6M[['Credit']], left_on='CID', right_index=True, how='left')
new_df = new_df.rename(columns={'Credit': 'Count_credit_trans_list_S6M'})

# Calculate Count_credit_trans_list_F6M per CID
credit_counts_F6M = count_transactions(df, 'CID', 'Transaction Type', '2022-01-01', '2022-06-30',date_format)
new_df = new_df.merge(credit_counts_F6M[['Credit']], left_on='CID', right_index=True, how='left')
new_df = new_df.rename(columns={'Credit': 'Count_credit_trans_list_T6M'})

# Calculate Count_credit_trans_list_L6M per CID
credit_counts_L6M = count_transactions(df, 'CID', 'Transaction Type', '2022-07-01', '2022-12-31', date_format)
new_df = new_df.merge(credit_counts_L6M[['Credit']], left_on='CID', right_index=True, how='left')
new_df = new_df.rename(columns={'Credit': 'Count_credit_trans_list_L6M'})

# Calculate General_Expense
general_expense = df[df['Narration'].apply(lambda x: x.split('_')[-1] if '_' in x else x)
                     .isin(["Grocery", "Pharmacy/Healthcare", "Wellness and Fitness", "Entertainment",
                            "Gasoline/Fuel", "Shopping", "Dining", "Ticket Booking", "Travel", "Electricity Bill",
                            "Water Bill", "Telephone Bill", "Gas Bill", "Internet Bill", "Cable Bill", "Merchant",
                            "Mobile Recharge"])].groupby('CID')['Amount'].sum()
new_df = new_df.merge(general_expense, left_on='CID', right_index=True, how='left')
new_df = new_df.rename(columns={'Amount': 'General_Expense_Total'})

# List of categories
categories = ["Grocery", "Pharmacy/Healthcare", "Wellness and Fitness", "Entertainment",
              "Gasoline/Fuel", "Shopping", "Dining", "Ticket Booking", "Travel", "Electricity Bill",
              "Water Bill", "Telephone Bill", "Gas Bill", "Internet Bill", "Cable Bill", "Merchant",
              "Mobile Recharge"]

# Filtering the DataFrame for the specified categories
filtered_df = df[df['Narration'].str.split('_').str[-1].isin(categories)]

# Calculating means for each category
means = filtered_df.groupby(['CID', filtered_df['Narration'].str.split('_').str[-1]])['Amount'].sum().reset_index()

# Creating pivot table with means for each category
pivot_means = means.pivot_table(index='CID', columns='Narration', values='Amount', aggfunc='first')

# Renaming columns to match the requested format
pivot_means.columns = ['General_Expense_' + col + '_Total' for col in pivot_means.columns]

# Merging the pivot table with the existing DataFrame 'new_df'
new_df = new_df.merge(pivot_means, on='CID', how='left')

# Calculate average_bank_balance_F6M (First 6 Months)
average_transaction_amount_F3M = df[df['Transaction Date'].dt.month <= 3].groupby('CID')['Amount'].rolling(window=180, min_periods=1).sum().reset_index(drop=True)
new_df['average_transaction_amount_F3M'] = average_transaction_amount_F3M

# Calculate and store rolling averages
for i, (start, end) in enumerate(date_ranges):
    col_name = f'Avg_Transaction_Amount_{start}_to_{end}'
    rolling_avg = rolling_avg_for_range(df, 'CID', 'Amount', start, end)
    new_df = new_df.merge(rolling_avg, left_on='CID', right_index=True, how='left', suffixes=('', f'_{col_name}'))

# Calculate average_bank_balance_L6M (Last 6 Months up to December 2022)
end_date = pd.to_datetime('2022-12-31')

new_df.fillna(0, inplace = True)

C:\Users\nitish.mallick\AppData\Local\Temp\ipykernel_944\728483500.py:30: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  mask = (df['Transaction Date'] >= start_date) & (df['Transaction Date'] <= end_date)
C:\Users\nitish.mallick\AppData\Local\Temp\ipykernel_944\728483500.py:30: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  mask = (df['Transaction Date'] >= start_date) & (df['Transaction Date'] <= end_date)
C:\Users\nitish.mallick\AppData\Local\Temp\ipykernel_944\728483500.py:30: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  mask = (df['Transaction Date'] >

In [60]:
new_df

,CID,Balance,Balance_rolling_avg_01-04-2021_to_30-06-2021,Balance_rolling_avg_01-07-2021_to_30-09-2021,Balance_rolling_avg_01-10-2021_to_31-12-2021,Balance_rolling_avg_01-01-2022_to_31-03-2022,Balance_rolling_avg_01-04-2022_to_30-06-2022,Balance_rolling_avg_01-07-2022_to_30-09-2022,Balance_rolling_avg_01-10-2022_to_31-12-2022,Count_minimum_balance_Total,...,General_Expense_Wellness and Fitness_Total,average_transaction_amount_F3M,Amount,Amount_Avg_Transaction_Amount_01-04-2021_to_30-06-2021,Amount_Avg_Transaction_Amount_01-07-2021_to_30-09-2021,Amount_Avg_Transaction_Amount_01-10-2021_to_31-12-2021,Amount_Avg_Transaction_Amount_01-01-2022_to_31-03-2022,Amount_Avg_Transaction_Amount_01-04-2022_to_30-06-2022,Amount_Avg_Transaction_Amount_01-07-2022_to_30-09-2022,Amount_Avg_Transaction_Amount_01-10-2022_to_31-12-2022
0,1,1199.506522,1521.118652,1534.303118,1565.854494,2460.559333,1818.192857,1831.141053,2005.842273,59,...,471.39,7702.38,749.676957,576.950787,691.827957,711.605618,3118.005333,2799.575357,2838.919737,2811.093182
1,2,2291.721013,2228.253015,2125.162910,1893.091275,2.376667,53.707273,70.304833,65.998281,121,...,869.00,7710.68,740.087975,620.170074,559.181493,503.962617,0.110000,22.117273,24.213667,22.704687
2,3,1756.186598,1135.924729,1344.611429,1393.452292,3095.966250,2073.177143,2044.076333,1858.321818,74,...,895.88,8183.74,547.670206,470.946667,457.242473,601.414844,3088.757500,2712.120000,2683.467667,2439.546667
3,4,2449.603030,1740.037982,1539.473474,1598.374656,1167.302069,651.635116,99.045349,77.266226,111,...,0.00,15424.94,472.770000,423.739035,461.356368,485.793092,482.967586,179.983256,25.516512,25.813585
4,5,1545.498302,1525.272895,1527.371429,1562.693561,1387.192727,1596.226842,1557.098529,1883.520833,60,...,866.31,17414.35,403.593585,486.097281,741.644841,902.941136,2311.660000,2673.752632,2614.750588,2671.629792
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18745,18746,2705.188158,2284.013750,1881.632750,1830.750521,18.272500,2749.633947,2263.109856,2307.650481,94,...,0.00,24426.23,504.346053,404.204722,488.417250,578.274479,0.317500,661.160132,607.664029,639.096096
18746,18747,1395.806753,1109.757023,1169.189691,1141.162262,2356.451250,2127.889048,1862.017083,1655.193704,56,...,2058.03,25067.94,603.658831,511.047481,590.569321,653.052738,1897.603750,2753.652857,2409.497083,2141.805926
18747,18748,2109.316512,1819.663944,1728.736545,1701.934524,2709.375000,1380.661667,1026.194706,753.442917,51,...,799.47,25068.25,525.838256,469.354789,488.709895,543.202857,2378.590000,1294.779167,913.742353,686.671250
18748,18749,960.311304,349.156606,373.444750,370.505785,404.737500,696.869231,800.138261,746.009655,83,...,668.35,25.03,332.513478,189.847064,256.135583,259.832645,637.718750,796.440769,897.079565,884.267241


In [62]:
new_df.to_csv('Loyal_Salaried_1.csv', index =False)

In [64]:
df.to_csv('At-Risk_Salaried_Part1.csv', index =False)

In [70]:
import pandas as pd
df = pd.read_csv('D:\All_Files\Savings_Account\\2.0\Augment\Medium_Business.csv')

In [71]:
new_df

,CID,Balance,Balance_rolling_avg_01-04-2021_to_30-06-2021,Balance_rolling_avg_01-07-2021_to_30-09-2021,Balance_rolling_avg_01-10-2021_to_31-12-2021,Balance_rolling_avg_01-01-2022_to_31-03-2022,Balance_rolling_avg_01-04-2022_to_30-06-2022,Balance_rolling_avg_01-07-2022_to_30-09-2022,Balance_rolling_avg_01-10-2022_to_31-12-2022,Count_minimum_balance_Total,...,average_transaction_amount_F3M,Amount,Amount_Avg_Transaction_Amount_01-04-2021_to_30-06-2021,Amount_Avg_Transaction_Amount_01-07-2021_to_30-09-2021,Amount_Avg_Transaction_Amount_01-10-2021_to_31-12-2021,Amount_Avg_Transaction_Amount_01-01-2022_to_31-03-2022,Amount_Avg_Transaction_Amount_01-04-2022_to_30-06-2022,Amount_Avg_Transaction_Amount_01-07-2022_to_30-09-2022,Amount_Avg_Transaction_Amount_01-10-2022_to_31-12-2022,Category
0,CUST_ID17691258,3088.465781,2386.986457,2095.316585,1966.061314,1838.487595,2071.558542,2967.071542,3281.221719,10.0,...,19.460000,164.648047,161.137829,172.175691,184.248750,187.970000,208.126806,196.022289,188.930271,Medium Pension
1,CUST_ID10494740,1686.692048,1594.082604,1836.158512,1938.263077,6178.990806,5182.642353,4631.321751,4116.853252,10.0,...,2506.785000,148.712530,151.729010,172.015331,186.773612,258.793065,240.967983,279.194294,296.186019,Medium Pension
2,CUST_ID14999054,2729.764681,2350.700508,5704.381818,10765.138385,19032.150000,17238.198168,17468.447500,18283.504953,9.0,...,1822.463333,284.445532,197.601864,280.367190,312.689271,504.777200,504.794504,498.213056,541.716085,Medium Pension
3,CUST_ID10191797,6113.577156,4532.033818,4938.277059,5568.527349,5702.915147,6464.551449,7361.404740,6893.143081,4.0,...,1482.305000,216.005046,200.689727,214.740441,256.211898,278.748971,255.949130,297.549884,386.916566,Medium Pension
4,CUST_ID17390477,6489.978993,9380.664052,12066.102945,18455.894626,52368.965345,63361.208049,73241.545979,78142.468065,0.0,...,1201.512000,237.191511,242.029009,298.323455,326.429483,539.607241,465.919431,491.933557,605.576912,Medium Pension
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4979,CUST_ID11993050,8717.162222,10990.077126,11025.734556,10878.733710,12465.580571,13640.881752,14603.882956,13053.365270,13.0,...,213.532167,441.595333,486.431609,472.386509,453.362379,381.207143,413.322847,412.849310,470.765394,Medium Pension
4980,CUST_ID17794680,8097.077742,12929.063125,17303.859101,19779.137238,37008.593051,42319.493846,46381.886651,50501.549821,0.0,...,213.727438,357.262903,267.844479,247.333652,255.677657,307.681356,265.133077,268.368837,284.386774,Medium Pension
4981,CUST_ID12995331,4123.407111,5086.363558,6091.705771,6018.862422,8399.547234,10344.289735,12173.075515,12897.507050,0.0,...,214.705410,235.915556,227.570962,209.726286,208.137148,221.104681,205.731947,182.707320,185.846590,Medium Pension
4982,CUST_ID12596555,6260.971800,7097.318673,6425.904595,6323.382713,3777.045870,4066.139455,4495.790229,4395.529004,4.0,...,213.087073,303.690200,367.123673,322.959730,317.807394,223.783261,285.717909,299.511086,299.878528,Medium Pension


In [69]:
new_df.to_csv('Medium_Pension_Derived.csv', index = False)

In [55]:
df

,CID,Transaction Date,Transaction Type,Amount,Narration,Balance,Category
0,CUST_ID17691258,2021-01-01,Credit,2021.98,BACS,5357.16,Medium Pension
1,CUST_ID17691258,2021-01-01,Credit,267.86,Interest,5625.02,Medium Pension
2,CUST_ID17691258,2021-01-02,Debit,31.50,Direct Debit_Autopay,5593.52,Medium Pension
3,CUST_ID17691258,2021-01-02,Credit,89.25,Faster Payment,5682.77,Medium Pension
4,CUST_ID17691258,2021-01-02,Debit,540.95,Standing Order_Rent,5141.82,Medium Pension
...,...,...,...,...,...,...,...
2805765,CUST_ID17894646,2023-01-27,Debit,150.71,Cash Withdrawal,39296.17,Medium Pension
2805766,CUST_ID17894646,2023-01-27,Debit,52.60,Fast Payment_Pharmacy/Healthcare,39243.57,Medium Pension
2805767,CUST_ID17894646,2023-01-28,Debit,45.26,Cash_Grocery,39198.31,Medium Pension
2805768,CUST_ID17894646,2023-01-29,Debit,18.18,Fast Payment_Travel,39180.13,Medium Pension


In [53]:
df1 = pd.read_csv('D:\All_Files\Savings_Account\\2.0\Augment\Medium_Business_Derived.csv')
df2 = pd.read_csv('D:\All_Files\Savings_Account\\2.0\Augment\Medium_Pension_Derived.csv')
df3 = pd.read_csv('D:\All_Files\Savings_Account\\2.0\Augment\Medium Salaried Derived.csv')
df4 = pd.read_csv('D:\All_Files\Savings_Account\\2.0\Augment\Loyal_Business_Derived.csv')
df5 = pd.read_csv('D:\All_Files\Savings_Account\\2.0\Augment\Loyal_Pension_Derived.csv')
df6 = pd.read_csv('D:\All_Files\Savings_Account\\2.0\Augment\Loyal_Salaried_Derived.csv')
df7 = pd.read_csv('D:\All_Files\Savings_Account\\2.0\Augment\At-Risk_Business_Derived.csv')
df8 = pd.read_csv('D:\All_Files\Savings_Account\\2.0\Augment\At-Risk_Pension_Derived.csv')
df9 = pd.read_csv('D:\All_Files\Savings_Account\\2.0\Augment\At-Risk_Salaried_Derived.csv')
# If the DataFrames have the same columns and you want to reset the index
result = pd.concat([df4,df5,df6,df1, df2, df3,df7,df8,df9], ignore_index=True)

result.to_csv('Savings.csv', index = False)

In [54]:
result

,CID,Balance,Balance_rolling_avg_01-04-2021_to_30-06-2021,Balance_rolling_avg_01-07-2021_to_30-09-2021,Balance_rolling_avg_01-10-2021_to_31-12-2021,Balance_rolling_avg_01-01-2022_to_31-03-2022,Balance_rolling_avg_01-04-2022_to_30-06-2022,Balance_rolling_avg_01-07-2022_to_30-09-2022,Balance_rolling_avg_01-10-2022_to_31-12-2022,Count_minimum_balance_Total,...,General_Expense_Wellness and Fitness_Total,average_transaction_amount_F3M,Amount,Amount_Avg_Transaction_Amount_01-04-2021_to_30-06-2021,Amount_Avg_Transaction_Amount_01-07-2021_to_30-09-2021,Amount_Avg_Transaction_Amount_01-10-2021_to_31-12-2021,Amount_Avg_Transaction_Amount_01-01-2022_to_31-03-2022,Amount_Avg_Transaction_Amount_01-04-2022_to_30-06-2022,Amount_Avg_Transaction_Amount_01-07-2022_to_30-09-2022,Amount_Avg_Transaction_Amount_01-10-2022_to_31-12-2022
0,CUST_ID18886910,14573.676250,36558.041892,39251.589468,49733.723204,34301.836400,67999.607571,72827.554869,64711.558476,0.0,...,126.927931,248.060000,3559.837500,2316.884595,1881.575106,1649.814369,1050.465067,1415.338750,1708.684791,1651.674990
1,CUST_ID17378656,967.139000,5177.223571,6395.929655,4744.074407,3617.540541,3696.674221,5579.091055,5599.246885,90.0,...,0.000000,370.443333,400.359000,1161.095476,1534.231494,1254.579605,896.662973,905.488961,1310.355316,1454.196585
2,CUST_ID19483117,1432.241250,11339.242778,37272.360303,56131.686868,11007.151724,9252.563468,7678.887971,6446.989737,65.0,...,73.323333,301.700000,310.616250,1556.398333,1587.491616,2323.292967,2081.328103,1931.307613,1835.286268,1563.688070
3,CUST_ID17189852,17318.822222,21676.488889,27290.171538,42040.246776,68912.217869,57344.561292,57783.344750,58610.327078,0.0,...,120.822308,245.690000,2291.645556,1371.912222,1394.800879,1470.207158,1832.482951,1408.137970,1708.569625,1664.613681
4,CUST_ID14284119,647.625000,2228.281379,7000.891507,4959.749344,4675.728585,6553.039211,6617.799878,5849.047240,82.0,...,0.000000,345.011667,287.100625,906.883103,1819.338082,1329.492787,1488.062547,1730.000526,1751.276531,1586.579058
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,CUST_ID18368702,860.407308,791.195484,771.691176,758.798829,491.632500,646.996250,544.683437,522.194865,52.0,...,116.590000,82.074500,263.827115,205.992473,230.144020,267.615946,283.425833,535.987500,498.588750,509.927027
99996,CUST_ID14248996,1922.927941,1252.714077,1179.859200,1227.961388,971.174286,982.298333,884.027000,1270.003548,56.0,...,125.025000,55.170000,457.079265,394.340462,392.137333,400.439282,1129.902857,918.555556,826.680500,1058.440968
99997,CUST_ID1381859,4399.703548,3715.165253,3625.220970,3627.108343,1699.492568,1035.757727,935.271622,796.089098,83.0,...,112.528182,1569.810000,749.334516,674.903232,790.945896,732.433204,380.610541,203.599273,168.731261,149.383684
99998,CUST_ID131984,13215.261053,20570.400167,19678.333421,18318.269415,1859.974211,1533.732581,1617.770000,1825.044643,25.0,...,138.645000,1252.393333,365.003895,358.368167,574.934474,676.676585,1614.089474,1489.514839,1779.762619,1872.173750


In [45]:
df2 = pd.read_csv('Loyal_Pension.csv')

In [107]:
new_df.to_csv('Loyal_Salaried.csv', index = False)

In [14]:
# Set the maximum number of rows to be displayed
pd.set_option('display.max_rows', 10000)  # Replace 10 with the desired number of rows

In [47]:
df2[df2['Transaction Type'] == 'Credit']

,CID,Transaction Date,Transaction Type,Amount,Narration,Balance
5,1,2021-01-22,Credit,3927.97,BACS,4279.96
7,1,2021-01-23,Credit,212.26,Interest,4457.54
10,1,2021-02-02,Credit,3927.97,BACS,8253.70
13,1,2021-02-09,Credit,357.26,Interest,7502.53
19,1,2021-03-08,Credit,3927.97,BACS,9665.54
...,...,...,...,...,...,...
2432099,2600,2022-12-12,Credit,178.18,Faster Payment,59672.40
2432101,2600,2022-12-13,Credit,277.73,Faster Payment,59944.51
2432106,2600,2022-12-17,Credit,294.02,Faster Payment,59887.64
2432141,2600,2023-01-02,Credit,3318.13,BACS,58147.49


In [48]:
df2['Transaction_date'].nunique()

KeyError: 'Transaction_date'

In [49]:
import pandas as pd
df = pd.read_csv('D:\All_Files\Savings_Account\\2.0\Augment\Loyal_Pension.csv')

In [67]:
filtered_df['CID'].nunique()

7490

In [50]:
# Convert 'Transaction Date' to datetime if it's not already in datetime format
df['Transaction Date'] = pd.to_datetime(df['Transaction Date'])

filtered_df = df[(df['Transaction Type'] == 'Credit') & (df['Narration'] == 'BACS') ]

# Adding 'Month' column to the filtered dataframe
filtered_df['Month'] = filtered_df['Transaction Date'].dt.to_period('M')

# Finding average monthly amount for each unique CID
monthly_avg_amounts = filtered_df.groupby(['CID', 'Month'])['Amount'].mean().groupby('CID').mean()

C:\Users\nitish.mallick\AppData\Local\Temp\ipykernel_23348\3745838880.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Month'] = filtered_df['Transaction Date'].dt.to_period('M')


In [52]:
monthly_avg_amounts

CID
1       3927.97
2       1372.48
3       1927.46
4       1514.74
5       3781.17
         ...   
2596    3510.80
2597    2834.15
2598    2791.54
2599    3343.30
2600    3318.13
Name: Amount, Length: 2600, dtype: float64

In [53]:
missing_values = set(range(1, 2601)) - set(monthly_avg_amounts.index)
missing_values

set()

In [54]:
monthly_avg_amounts.max()

4999.48

In [81]:
import numpy as np

# New indices and corresponding values
indices = [986, 1881, 4522, 4884, 5163, 5367, 5520, 5546, 5584, 5848]
random_values = np.random.randint(1600, 14001, size=len(indices))

# Create a new Series with random values and specified indices
new_values = pd.Series(random_values, index=indices)

# Concatenate the two Series
monthly_avg_amounts = pd.concat([monthly_avg_amounts, new_values])


In [55]:
import pandas as pd
df1 = pd.read_csv('D:\All_Files\Savings_Account\\2.0\Augment\Loyal_Pension_Derived.csv')

In [56]:
df1['Monthly_Income_Savings'] = 0

In [57]:
# Assuming you have a dataframe named 'your_dataframe' containing the data
df1['Monthly_Income_Savings'] = monthly_avg_amounts.values

In [59]:
monthly_avg_amounts.values

array([3927.97, 1372.48, 1927.46, ..., 2791.54, 3343.3 , 3318.13])

In [58]:
df1['Monthly_Income_Savings']

0       3927.97
1       1372.48
2       1927.46
3       1514.74
4       3781.17
         ...   
2595    3510.80
2596    2834.15
2597    2791.54
2598    3343.30
2599    3318.13
Name: Monthly_Income_Savings, Length: 2600, dtype: float64

In [136]:
df1[df1['Monthly_Income_Savings'] == 0]

,CID,Balance,Balance_rolling_avg_01-04-2021_to_30-06-2021,Balance_rolling_avg_01-07-2021_to_30-09-2021,Balance_rolling_avg_01-10-2021_to_31-12-2021,Balance_rolling_avg_01-01-2022_to_31-03-2022,Balance_rolling_avg_01-04-2022_to_30-06-2022,Balance_rolling_avg_01-07-2022_to_30-09-2022,Balance_rolling_avg_01-10-2022_to_31-12-2022,Count_minimum_balance_Total,...,Amount,Amount_Avg_Transaction_Amount_01-04-2021_to_30-06-2021,Amount_Avg_Transaction_Amount_01-07-2021_to_30-09-2021,Amount_Avg_Transaction_Amount_01-10-2021_to_31-12-2021,Amount_Avg_Transaction_Amount_01-01-2022_to_31-03-2022,Amount_Avg_Transaction_Amount_01-04-2022_to_30-06-2022,Amount_Avg_Transaction_Amount_01-07-2022_to_30-09-2022,Amount_Avg_Transaction_Amount_01-10-2022_to_31-12-2022,Category,Monthly_Income_Savings


In [60]:
df1.to_csv('Loyal_Pension_Derived.csv', index = False)

In [62]:
savings = pd.read_csv('D:\All_Files\Savings_Account\\2.0\Augment\Savings_Derived.csv')

In [64]:
savings = savings[savings['Category'] != 'Loyal Pension']

In [65]:
savings

,CID,Balance,Balance_rolling_avg_01-04-2021_to_30-06-2021,Balance_rolling_avg_01-07-2021_to_30-09-2021,Balance_rolling_avg_01-10-2021_to_31-12-2021,Balance_rolling_avg_01-01-2022_to_31-03-2022,Balance_rolling_avg_01-04-2022_to_30-06-2022,Balance_rolling_avg_01-07-2022_to_30-09-2022,Balance_rolling_avg_01-10-2022_to_31-12-2022,Count_minimum_balance_Total,...,Amount,Amount_Avg_Transaction_Amount_01-04-2021_to_30-06-2021,Amount_Avg_Transaction_Amount_01-07-2021_to_30-09-2021,Amount_Avg_Transaction_Amount_01-10-2021_to_31-12-2021,Amount_Avg_Transaction_Amount_01-01-2022_to_31-03-2022,Amount_Avg_Transaction_Amount_01-04-2022_to_30-06-2022,Amount_Avg_Transaction_Amount_01-07-2022_to_30-09-2022,Amount_Avg_Transaction_Amount_01-10-2022_to_31-12-2022,Category,Monthly_Income_Savings
0,1000001,14573.676250,36558.041890,39251.589470,49733.723200,34301.836400,67999.607570,72827.554870,64711.558480,0,...,3559.837500,2316.884595,1881.575106,1649.814369,1050.465067,1415.338750,1708.684791,1651.674990,Loyal Business,8538.204910
1,1000002,12458.892350,29701.509290,32764.586630,49544.293270,23059.958250,33745.290180,40529.396050,45421.182750,0,...,1428.073529,1629.278333,1360.976211,1525.657970,1270.519612,1245.721327,1265.601867,1373.203747,Loyal Business,9212.229389
2,1000003,1715.065000,12380.019320,24418.109550,45223.428560,87343.521000,72131.871840,73478.580050,75201.235880,0,...,127.581667,1073.085909,1269.972614,1212.456051,2268.190727,1740.961749,1860.892356,1850.673964,Loyal Business,9238.743117
3,1000004,967.139000,5177.223571,6395.929655,4744.074407,3617.540541,3696.674221,5579.091055,5599.246885,90,...,400.359000,1161.095476,1534.231494,1254.579605,896.662973,905.488961,1310.355316,1454.196585,Loyal Business,8122.398130
4,1000005,1978.103000,5769.430435,29163.155850,51935.832050,64781.767860,56975.955000,56152.353770,64502.964390,0,...,912.275000,633.587391,1166.136064,2031.232923,1883.323932,1662.359962,1559.452592,1557.698898,Loyal Business,8992.601808
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112495,1112496,860.407308,791.195484,771.691176,758.798829,491.632500,646.996250,544.683437,522.194865,52,...,263.827115,205.992473,230.144020,267.615946,283.425833,535.987500,498.588750,509.927027,At-Risk Salaried,1538.080000
112496,1112497,1922.927941,1252.714077,1179.859200,1227.961388,971.174286,982.298333,884.027000,1270.003548,56,...,457.079265,394.340462,392.137333,400.439282,1129.902857,918.555556,826.680500,1058.440968,At-Risk Salaried,3894.330000
112497,1112498,4399.703548,3715.165253,3625.220970,3627.108343,1699.492568,1035.757727,935.271622,796.089098,83,...,749.334516,674.903232,790.945896,732.433204,380.610541,203.599273,168.731261,149.383684,At-Risk Salaried,6289.640000
112498,1112499,13215.261050,20570.400170,19678.333420,18318.269410,1859.974211,1533.732581,1617.770000,1825.044643,25,...,365.003895,358.368167,574.934474,676.676585,1614.089474,1489.514839,1779.762619,1872.173750,At-Risk Salaried,7108.580000


In [66]:
savings = pd.concat([savings,df1])

In [67]:
savings

,CID,Balance,Balance_rolling_avg_01-04-2021_to_30-06-2021,Balance_rolling_avg_01-07-2021_to_30-09-2021,Balance_rolling_avg_01-10-2021_to_31-12-2021,Balance_rolling_avg_01-01-2022_to_31-03-2022,Balance_rolling_avg_01-04-2022_to_30-06-2022,Balance_rolling_avg_01-07-2022_to_30-09-2022,Balance_rolling_avg_01-10-2022_to_31-12-2022,Count_minimum_balance_Total,...,Amount,Amount_Avg_Transaction_Amount_01-04-2021_to_30-06-2021,Amount_Avg_Transaction_Amount_01-07-2021_to_30-09-2021,Amount_Avg_Transaction_Amount_01-10-2021_to_31-12-2021,Amount_Avg_Transaction_Amount_01-01-2022_to_31-03-2022,Amount_Avg_Transaction_Amount_01-04-2022_to_30-06-2022,Amount_Avg_Transaction_Amount_01-07-2022_to_30-09-2022,Amount_Avg_Transaction_Amount_01-10-2022_to_31-12-2022,Category,Monthly_Income_Savings
0,1000001,14573.676250,36558.041890,39251.589470,49733.723200,34301.836400,67999.607570,72827.554870,64711.558480,0,...,3559.837500,2316.884595,1881.575106,1649.814369,1050.465067,1415.338750,1708.684791,1651.674990,Loyal Business,8538.204910
1,1000002,12458.892350,29701.509290,32764.586630,49544.293270,23059.958250,33745.290180,40529.396050,45421.182750,0,...,1428.073529,1629.278333,1360.976211,1525.657970,1270.519612,1245.721327,1265.601867,1373.203747,Loyal Business,9212.229389
2,1000003,1715.065000,12380.019320,24418.109550,45223.428560,87343.521000,72131.871840,73478.580050,75201.235880,0,...,127.581667,1073.085909,1269.972614,1212.456051,2268.190727,1740.961749,1860.892356,1850.673964,Loyal Business,9238.743117
3,1000004,967.139000,5177.223571,6395.929655,4744.074407,3617.540541,3696.674221,5579.091055,5599.246885,90,...,400.359000,1161.095476,1534.231494,1254.579605,896.662973,905.488961,1310.355316,1454.196585,Loyal Business,8122.398130
4,1000005,1978.103000,5769.430435,29163.155850,51935.832050,64781.767860,56975.955000,56152.353770,64502.964390,0,...,912.275000,633.587391,1166.136064,2031.232923,1883.323932,1662.359962,1559.452592,1557.698898,Loyal Business,8992.601808
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2595,2596,2957.237910,2770.348455,3837.783074,5899.426037,18630.836430,25921.268210,31305.401550,41808.971110,2,...,218.017463,207.071273,224.598479,236.905958,453.098333,373.874464,395.895000,343.605882,Loyal Pension,3510.800000
2596,2597,2026.905726,2026.614130,2181.827164,3764.886860,9219.806829,13923.784580,16824.413770,23289.939770,3,...,184.468803,197.651630,201.481828,208.627080,374.340976,280.031589,293.152963,252.224631,Loyal Pension,2834.150000
2597,2598,9399.204127,10141.575370,11238.514590,13083.411560,25542.850000,33624.614200,37357.300410,47189.179200,0,...,173.483280,184.300071,184.751541,199.882594,499.451563,373.855800,398.466069,318.368400,Loyal Pension,2791.540000
2598,2599,5090.248289,5939.468870,7683.713775,10253.478420,25929.246410,36709.879920,40719.913900,50326.580550,0,...,165.176184,166.609916,177.635591,194.606927,488.632051,318.350492,384.161585,322.573955,Loyal Pension,3343.300000


In [68]:
savings.to_csv('Savings_Derived.csv', index = False)

In [45]:
df1['Category'].value_counts()

Category
Medium Salaried      35000
Loyal Salaried       18750
At-Risk Salaried     18750
Medium Business      15000
Loyal Business        7500
At Risk Business      7500
Medium Pension        5000
Loyal Pension         2500
At-Risk Pension       2500
Name: count, dtype: int64

In [94]:
df2 = pd.read_csv('D:\All_Files\Base_Data\\2.0\Base_Data_1Lakh.csv')

In [140]:
df2.columns

Index(['CID', 'Total_bank_products', 'Products', 'CREDIT_SCORE',
       'CUSTOMER_RATING', 'Gender', 'Age', 'Num_Dependents', 'PinCode_rating',
       'IsOnlineBankingUser', 'IsSmsBankingUser', 'IsChequeBookUser',
       'Organisation_type', 'IscurrentAddressDiffFromPermanentAddress',
       'IsUpiCustomer', 'Marital_Status', 'Recently_Married', 'tier_city',
       'Monthly_Income', 'Is_locker_user', 'Is_app_user', 'Total_Experience',
       'months_current_job', 'Existing_Customer_Since_When',
       'Type_of_employment', 'Education', 'Is_offline_customer',
       'Is_hybrid_customer', 'Type_of_Residence', 'Years_of_Residence',
       'Is_Life_Insurance_Holder', 'Is_Medical_Insurance_Holder',
       'Is_Metropolitan_customer', 'Pincode', 'Has_Standard_Credit_Card',
       'Has_Premium_Credit_Card', 'Has_Personal_Loan', 'Customer_Type'],
      dtype='object')

In [141]:
df2['Type_of_employment'].value_counts()

Type_of_employment
Salaried     69795
Business     20133
Pensioner    10072
Name: count, dtype: int64

In [3]:
import pandas as pd

In [93]:
df1 = pd.read_csv('D:\All_Files\Savings_Account\\2.0\Augment\Savings_Derived.csv')

In [120]:
salaried = df2[df2['Type_of_employment'] == 'Pensioner']
neft_df1 = df1[(df1['Category'] == 'Loyal Pension') | (df1['Category'] == 'Medium Pension') | (df1['Category'] == 'At-Risk Pension')]

In [122]:
salaried

,CID,Total_bank_products,Products,CREDIT_SCORE,CUSTOMER_RATING,Gender,Age,Num_Dependents,PinCode_rating,IsOnlineBankingUser,...,Type_of_Residence,Years_of_Residence,Is_Life_Insurance_Holder,Is_Medical_Insurance_Holder,Is_Metropolitan_customer,Pincode,Has_Standard_Credit_Card,Has_Premium_Credit_Card,Has_Personal_Loan,Customer_Type
22510,22511,2,"['Savings Account', 'Standard Credit Card']",528,2.8,F,64,4,3,No,...,Owned,44,Yes,Yes,Yes,644724,1,0,0,Loyal
22511,22512,2,"['Savings Account', 'Personal Loan']",652,1.9,F,72,4,4,No,...,Owned,51,Yes,Yes,Yes,164158,0,0,1,Loyal
22512,22513,3,"['Savings Account', 'Standard Credit Card', 'P...",694,2.5,F,84,3,0,No,...,Rented with Agreement,64,Yes,Yes,No,707480,1,0,1,Loyal
22513,22514,4,"['Savings Account', 'Premium Credit Card', 'St...",510,2.8,M,85,0,9,Yes,...,Owned,66,Yes,Yes,Yes,625440,1,1,1,Loyal
22514,22515,3,"['Savings Account', 'Premium Credit Card', 'St...",631,2.8,M,88,4,0,Yes,...,Owned,67,Yes,Yes,Yes,254427,1,1,0,Loyal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77442,77443,4,"['Savings Account', 'Premium Credit Card', 'St...",717,1.9,M,88,2,1,No,...,Owned,67,Yes,Yes,No,164686,1,1,1,At-Risk
77443,77444,1,['Savings Account'],743,2.2,F,85,1,2,No,...,Owned,64,Yes,Yes,No,708728,0,0,0,At-Risk
77444,77445,3,"['Savings Account', 'Standard Credit Card', 'P...",793,2.2,F,61,4,4,Yes,...,Owned,41,Yes,Yes,No,712309,1,0,1,At-Risk
77445,77446,4,"['Savings Account', 'Premium Credit Card', 'St...",745,1.9,M,65,4,2,Yes,...,Owned,45,Yes,Yes,Yes,920086,1,1,1,At-Risk


In [123]:
neft_df1_copy = neft_df1.copy()

In [124]:
neft_df1_copy['Base_CID'] = 0

In [125]:
for _, row in salaried.iterrows():
    # Calculate the absolute difference between 'Monthly_Salary' and 'Monthly_Income_Savings'
    neft_df1['diff'] = abs(neft_df1['Monthly_Income_Savings'] - row['Monthly_Income'])
    
    # Find the row in neft_df1 with the minimum difference
    min_CID_index = neft_df1['diff'].idxmin()  # Get index of the row with minimum difference
    
    # Update the 'CID' value in neft_df1 with the corresponding value from the row DataFrame
    neft_df1_copy.at[min_CID_index, 'Base_CID'] = row['CID']   
    
    neft_df1 = neft_df1[neft_df1.index != min_CID_index]

C:\Users\nitish.mallick\AppData\Local\Temp\ipykernel_23348\1603731059.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  neft_df1['diff'] = abs(neft_df1['Monthly_Income_Savings'] - row['Monthly_Income'])
C:\Users\nitish.mallick\AppData\Local\Temp\ipykernel_23348\1603731059.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  neft_df1['diff'] = abs(neft_df1['Monthly_Income_Savings'] - row['Monthly_Income'])


In [113]:
neft_df1_copy[neft_df1_copy['Base_CID'] == 0]

,CID,Balance,Balance_rolling_avg_01-04-2021_to_30-06-2021,Balance_rolling_avg_01-07-2021_to_30-09-2021,Balance_rolling_avg_01-10-2021_to_31-12-2021,Balance_rolling_avg_01-01-2022_to_31-03-2022,Balance_rolling_avg_01-04-2022_to_30-06-2022,Balance_rolling_avg_01-07-2022_to_30-09-2022,Balance_rolling_avg_01-10-2022_to_31-12-2022,Count_minimum_balance_Total,...,Amount_Avg_Transaction_Amount_01-04-2021_to_30-06-2021,Amount_Avg_Transaction_Amount_01-07-2021_to_30-09-2021,Amount_Avg_Transaction_Amount_01-10-2021_to_31-12-2021,Amount_Avg_Transaction_Amount_01-01-2022_to_31-03-2022,Amount_Avg_Transaction_Amount_01-04-2022_to_30-06-2022,Amount_Avg_Transaction_Amount_01-07-2022_to_30-09-2022,Amount_Avg_Transaction_Amount_01-10-2022_to_31-12-2022,Category,Monthly_Income_Savings,Base_CID
7504,1007505,13120.715500,26183.725060,38748.892630,51155.582200,51726.34269,52290.02830,58225.00658,58595.67364,0,...,776.963563,917.990855,649.624167,570.415604,503.855167,415.706439,459.855270,Loyal Salaried,1165.74,0
7526,1007527,17951.900370,32601.581810,41894.477870,48371.072140,41654.91639,50675.53661,53107.14934,54828.03529,0,...,944.197361,967.079000,837.095250,429.363742,428.912232,409.979940,442.987348,Loyal Salaried,1194.34,0
7534,1007535,24475.172670,44234.533520,63631.205200,68793.092600,97559.90706,86180.36013,73248.57272,71081.39082,0,...,1478.589437,1368.806149,1015.382604,833.930882,885.781438,792.369956,695.145651,Loyal Salaried,1416.56,0
7542,1007543,28721.815410,50718.879880,80320.590850,97105.906790,176574.63820,181846.65830,155245.43880,139552.42350,0,...,1427.252099,1397.408727,1157.644613,713.852789,1110.567533,1167.412545,969.057917,Loyal Salaried,1152.50,0
7547,1007548,7972.468649,14317.260480,19804.855250,23357.244540,20124.60093,17666.97173,16122.08945,13532.20203,13,...,439.208795,461.308608,363.448170,264.226539,227.198054,244.759849,232.226772,Loyal Salaried,1178.43,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26229,1026230,6157.805217,7414.142869,12679.445100,16654.718160,47222.03478,66692.74366,72386.88552,73242.76867,0,...,303.326920,327.008991,366.455413,1360.146957,445.970131,512.593881,589.808123,Loyal Salaried,1165.28,0
26233,1026234,3737.647541,3015.259892,3386.593694,3837.728387,10830.64800,15073.40000,17317.44814,17304.16900,3,...,201.927297,194.019013,204.898898,514.250667,285.795701,253.895376,278.080824,Loyal Salaried,1147.57,0
26234,1026235,17372.374700,25255.433240,51129.885970,66411.593380,205835.17520,276024.02950,273060.76390,277724.43370,0,...,571.340798,563.307054,667.508808,2700.448387,1318.858289,1469.449966,1572.983112,Loyal Salaried,1381.85,0
26237,1026238,19015.279050,25124.232040,44677.603420,54624.893630,124973.98900,140686.69810,134326.46020,137871.04110,0,...,424.242833,434.052063,569.851918,1817.372069,1419.560392,900.521278,1035.659615,Loyal Salaried,1104.63,0


In [126]:
neft_df1_copy.to_csv('Pension_Derived_with_base_CID.csv', index = False)

In [82]:
# Assuming neft_df1 and neft_df2 are your DataFrames
merged_df = pd.merge(salaried, neft_df1_copy, on='CID', how='inner')

In [83]:
merged_df

,CID,Total_bank_products,Products,CREDIT_SCORE,CUSTOMER_RATING,Gender,Age,Num_Dependents,PinCode_rating,IsOnlineBankingUser,...,Amount,Amount_Avg_Transaction_Amount_01-04-2021_to_30-06-2021,Amount_Avg_Transaction_Amount_01-07-2021_to_30-09-2021,Amount_Avg_Transaction_Amount_01-10-2021_to_31-12-2021,Amount_Avg_Transaction_Amount_01-01-2022_to_31-03-2022,Amount_Avg_Transaction_Amount_01-04-2022_to_30-06-2022,Amount_Avg_Transaction_Amount_01-07-2022_to_30-09-2022,Amount_Avg_Transaction_Amount_01-10-2022_to_31-12-2022,Category,Monthly_Income_Savings
0,22511,2,"['Savings Account', 'Standard Credit Card']",528,2.8,F,64,4,3,No,...,401.178312,398.353769,409.114340,407.534504,594.104156,607.170781,642.727103,677.584464,Medium Pension,3677.92
1,22512,2,"['Savings Account', 'Personal Loan']",652,1.9,F,72,4,4,No,...,291.047073,277.124568,285.538353,258.447288,226.442209,296.260777,274.855299,275.643813,Loyal Pension,4005.67
2,22513,3,"['Savings Account', 'Standard Credit Card', 'P...",694,2.5,F,84,3,0,No,...,266.093784,224.662143,246.554946,339.661651,708.124839,825.343704,853.700641,849.439703,At-Risk Pension,3484.34
3,22514,4,"['Savings Account', 'Premium Credit Card', 'St...",510,2.8,M,85,0,9,Yes,...,215.220692,248.340693,286.531169,333.334479,215.670520,191.113105,196.335768,207.968630,Loyal Pension,3630.49
4,22515,3,"['Savings Account', 'Premium Credit Card', 'St...",631,2.8,M,88,4,0,Yes,...,337.469423,325.249247,355.432000,372.860973,423.740313,399.712199,391.039317,442.630302,Medium Pension,3743.36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10067,77443,4,"['Savings Account', 'Premium Credit Card', 'St...",717,1.9,M,88,2,1,No,...,1037.830000,722.816952,535.392682,478.553675,734.137031,641.728347,661.566198,614.832548,Medium Pension,4954.80
10068,77444,1,['Savings Account'],743,2.2,F,85,1,2,No,...,676.032069,840.269367,887.912326,873.236782,1695.917222,1591.248462,1559.459649,1574.424179,At-Risk Pension,4954.94
10069,77445,3,"['Savings Account', 'Standard Credit Card', 'P...",793,2.2,F,61,4,4,Yes,...,469.653710,544.283475,476.557454,563.488292,559.545645,619.839508,689.492867,836.572164,Medium Pension,4956.44
10070,77446,4,"['Savings Account', 'Premium Credit Card', 'St...",745,1.9,M,65,4,2,Yes,...,968.091429,725.599039,694.885858,654.390830,818.893585,727.314453,601.182920,463.769530,Medium Pension,4956.45


In [84]:
neft_df1_copy['CID'].nunique()

10100

In [38]:
merged_df

,CID,Total_bank_products,Products,CREDIT_SCORE,CUSTOMER_RATING,Gender,Age,Num_Dependents,PinCode_rating,IsOnlineBankingUser,...,Amount,Amount_Avg_Transaction_Amount_01-04-2021_to_30-06-2021,Amount_Avg_Transaction_Amount_01-07-2021_to_30-09-2021,Amount_Avg_Transaction_Amount_01-10-2021_to_31-12-2021,Amount_Avg_Transaction_Amount_01-01-2022_to_31-03-2022,Amount_Avg_Transaction_Amount_01-04-2022_to_30-06-2022,Amount_Avg_Transaction_Amount_01-07-2022_to_30-09-2022,Amount_Avg_Transaction_Amount_01-10-2022_to_31-12-2022,Category,Monthly_Income_Savings
0,17387,3,"['Savings Account', 'Standard Credit Card', 'P...",789,2.2,F,46,4,6,Yes,...,1176.533846,1265.673077,760.253066,780.873193,1969.961250,1351.922000,909.521750,967.344717,At Risk Business,3834.085000
1,17388,2,"['Savings Account', 'Premium Credit Card']",744,1.9,F,52,4,6,Yes,...,1190.394348,1506.835634,1464.865890,1081.574660,3269.438000,1972.608269,1429.404815,1302.857222,At Risk Business,4989.248333
2,17389,3,"['Savings Account', 'Premium Credit Card', 'St...",731,1.9,F,47,4,1,No,...,882.220112,631.512903,601.208955,598.588693,45.513636,52.752941,64.841154,63.261429,At Risk Business,4708.086333
3,17390,1,['Savings Account'],766,1.3,M,34,4,1,No,...,1026.427324,673.338768,559.466104,526.903171,89.680000,85.345000,183.661667,166.127436,At Risk Business,3988.576786
4,17391,3,"['Savings Account', 'Premium Credit Card', 'Pe...",717,1.9,M,48,4,0,Yes,...,812.918222,757.715542,702.460392,593.839074,70.639091,753.774762,654.751707,884.874694,At Risk Business,4243.866000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20128,82447,2,"['Savings Account', 'Premium Credit Card']",742,2.4,M,32,1,3,No,...,687.531111,874.185512,1003.451417,967.040066,1254.368571,1168.743810,1113.409426,1162.627132,Medium Business,9584.661818
20129,82448,2,"['Savings Account', 'Personal Loan']",801,2.8,M,29,3,5,Yes,...,1002.870000,2598.336909,1372.232381,1122.303345,656.651600,420.194336,724.946577,758.260245,Medium Business,9584.560385
20130,82449,3,"['Savings Account', 'Premium Credit Card', 'St...",799,3.3,M,59,2,0,No,...,724.055938,856.019216,1079.487656,1372.138378,2288.026444,2449.313232,2288.340977,2240.476429,Medium Business,9583.747549
20131,82450,2,"['Savings Account', 'Premium Credit Card']",702,1.8,F,31,4,0,Yes,...,472.449275,812.494508,1073.375838,1083.009283,2473.843500,1420.375900,1219.382113,1378.079238,Medium Business,9583.501107


In [85]:
merged_df.to_csv('Pension_with_Base.csv', index = False)

In [86]:
df = pd.read_csv('D:\All_Files\Savings_Account\\2.0\Augment\Savings_with_Base.csv')
df1 = pd.read_csv('D:\All_Files\Savings_Account\\2.0\Augment\Business_with_Base.csv')
df2 = pd.read_csv('D:\All_Files\Savings_Account\\2.0\Augment\Pension_with_Base.csv')

merged_df = pd.concat([df,df1,df2])

In [88]:
merged_df.to_csv('Base-Savings_Connected.csv', index = False)

In [16]:
concatenated_df

,99999,14886
CID,24998,4887
Total_bank_products,2,NaN
Products,"['Savings Account', 'Personal Loan']",NaN
CREDIT_SCORE,501,NaN
CUSTOMER_RATING,1.6,NaN
...,...,...
Amount_Avg_Transaction_Amount_01-07-2022_to_30-09-2022,NaN,840.380848
Amount_Avg_Transaction_Amount_01-10-2022_to_31-12-2022,NaN,835.476944
Category,NaN,Loyal Salaried
Monthly_Income_Savings,NaN,3022.01


In [12]:
final_result = pd.DataFrame({i: df.values.flatten() for i, df in enumerate(result_df)})

In [17]:
result_df

[                                                                                                0      \
 CID                                                                                                 1   
 Total_bank_products                                                                                 3   
 Products                                            ['Savings Account', 'Standard Credit Card', 'P...   
 CREDIT_SCORE                                                                                      812   
 CUSTOMER_RATING                                                                                   1.9   
 ...                                                                                               ...   
 Amount_Avg_Transaction_Amount_01-07-2022_to_30-...                                                NaN   
 Amount_Avg_Transaction_Amount_01-10-2022_to_31-...                                                NaN   
 Category                                     

In [13]:
final_result

,0,1,2,3,4,5,6,7,8,9,...,69785,69786,69787,69788,69789,69790,69791,69792,69793,69794
0,1,6,7,10,11,12,13,14,15,16,...,24988,24989,24990,24991,24992,24993,24995,24996,24997,24998
1,19665,4327,11277,8117,4286,946,7164,682,6327,861,...,1422,6546,301,1714,27271,4768,15178,3104,3665,4887
2,3,3,3,2,4,3,3,3,3,3,...,3,1,3,1,3,2,2,3,2,2
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"['Savings Account', 'Standard Credit Card', 'P...","['Savings Account', 'Premium Credit Card', 'Pe...","['Savings Account', 'Premium Credit Card', 'Pe...","['Savings Account', 'Standard Credit Card']","['Savings Account', 'Premium Credit Card', 'St...","['Savings Account', 'Premium Credit Card', 'St...","['Savings Account', 'Standard Credit Card', 'P...","['Savings Account', 'Premium Credit Card', 'St...","['Savings Account', 'Premium Credit Card', 'Pe...","['Savings Account', 'Standard Credit Card', 'P...",...,"['Savings Account', 'Standard Credit Card', 'P...",['Savings Account'],"['Savings Account', 'Premium Credit Card', 'Pe...",['Savings Account'],"['Savings Account', 'Premium Credit Card', 'Pe...","['Savings Account', 'Premium Credit Card']","['Savings Account', 'Personal Loan']","['Savings Account', 'Premium Credit Card', 'St...","['Savings Account', 'Personal Loan']","['Savings Account', 'Personal Loan']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,Medium Salaried,Loyal Salaried,Loyal Salaried,Loyal Salaried,Loyal Salaried,Loyal Salaried,Medium Salaried,Loyal Salaried,At-Risk Salaried,Loyal Salaried,...,Medium Salaried,At-Risk Salaried,At-Risk Salaried,At-Risk Salaried,Medium Salaried,Loyal Salaried,Medium Salaried,Loyal Salaried,Medium Salaried,Loyal Salaried
144,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
145,2309.43,2668.28,3490.07,2374.84,3847.86,3386.05,2308.88,2806.28,2806.37,2511.11,...,5071.35,7229.76,4910.57,7067.4,4689.07,3049.43,7355.83,3706.68,6134.2,3022.01
146,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:

final_result = pd.concat(result_df, axis=0) 
final_result

In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('D:\All_Files\Savings_Account\\2.0\Augment\Business_Derived_with_base_CID.csv')
df1 = pd.read_csv('D:\All_Files\Savings_Account\\2.0\Augment\Salaried_Derived_with_base_CID.csv')
df2 = pd.read_csv('D:\All_Files\Savings_Account\\2.0\Augment\Pension_Derived_with_base_CID.csv')

In [5]:
df2

,CID,Balance,Balance_rolling_avg_01-04-2021_to_30-06-2021,Balance_rolling_avg_01-07-2021_to_30-09-2021,Balance_rolling_avg_01-10-2021_to_31-12-2021,Balance_rolling_avg_01-01-2022_to_31-03-2022,Balance_rolling_avg_01-04-2022_to_30-06-2022,Balance_rolling_avg_01-07-2022_to_30-09-2022,Balance_rolling_avg_01-10-2022_to_31-12-2022,Count_minimum_balance_Total,...,Amount_Avg_Transaction_Amount_01-04-2021_to_30-06-2021,Amount_Avg_Transaction_Amount_01-07-2021_to_30-09-2021,Amount_Avg_Transaction_Amount_01-10-2021_to_31-12-2021,Amount_Avg_Transaction_Amount_01-01-2022_to_31-03-2022,Amount_Avg_Transaction_Amount_01-04-2022_to_30-06-2022,Amount_Avg_Transaction_Amount_01-07-2022_to_30-09-2022,Amount_Avg_Transaction_Amount_01-10-2022_to_31-12-2022,Category,Monthly_Income_Savings,Base_CID
0,1041251,3088.465781,2386.986457,2095.316585,1966.061314,1838.487595,2071.558542,2967.071542,3281.221719,10,...,161.137829,172.175691,184.248750,187.970000,208.126806,196.022289,188.930272,Medium Pension,2021.98,23841
1,1041252,1686.692048,1594.082604,1836.158512,1938.263077,6178.990806,5182.642353,4631.321751,4116.853252,10,...,151.729010,172.015331,186.773612,258.793065,240.967983,279.194294,296.186019,Medium Pension,1971.44,27323
2,1041253,2729.764681,2350.700508,5704.381818,10765.138390,19032.150000,17238.198170,17468.447500,18283.504950,9,...,197.601864,280.367190,312.689271,504.777200,504.794504,498.213056,541.716085,Medium Pension,3561.46,76502
3,1041254,6113.577156,4532.033818,4938.277059,5568.527349,5702.915147,6464.551449,7361.404740,6893.143081,4,...,200.689727,214.740441,256.211898,278.748971,255.949130,297.549884,386.916566,Medium Pension,2452.63,24380
4,1041255,6489.978993,9380.664052,12066.102950,18455.894630,52368.965340,63361.208050,73241.545980,78142.468060,0,...,242.029009,298.323455,326.429483,539.607241,465.919431,491.933557,605.576912,Medium Pension,4315.75,23060
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10095,1112596,2957.237910,2770.348455,3837.783074,5899.426037,18630.836430,25921.268210,31305.401550,41808.971110,2,...,207.071273,224.598479,236.905958,453.098333,373.874464,395.895000,343.605882,Loyal Pension,3510.80,25273
10096,1112597,2026.905726,2026.614130,2181.827164,3764.886860,9219.806829,13923.784580,16824.413770,23289.939770,3,...,197.651630,201.481828,208.627080,374.340976,280.031589,293.152963,252.224631,Loyal Pension,2834.15,24853
10097,1112598,9399.204127,10141.575370,11238.514590,13083.411560,25542.850000,33624.614200,37357.300410,47189.179200,0,...,184.300071,184.751541,199.882594,499.451563,373.855800,398.466069,318.368400,Loyal Pension,2791.54,28923
10098,1112599,5090.248289,5939.468870,7683.713775,10253.478420,25929.246410,36709.879920,40719.913900,50326.580550,0,...,166.609916,177.635591,194.606927,488.632051,318.350492,384.161585,322.573955,Loyal Pension,3343.30,25951


In [6]:

merged_df = pd.concat([df,df1,df2], axis=0) 
merged_df

,CID,Balance,Balance_rolling_avg_01-04-2021_to_30-06-2021,Balance_rolling_avg_01-07-2021_to_30-09-2021,Balance_rolling_avg_01-10-2021_to_31-12-2021,Balance_rolling_avg_01-01-2022_to_31-03-2022,Balance_rolling_avg_01-04-2022_to_30-06-2022,Balance_rolling_avg_01-07-2022_to_30-09-2022,Balance_rolling_avg_01-10-2022_to_31-12-2022,Count_minimum_balance_Total,...,Amount_Avg_Transaction_Amount_01-04-2021_to_30-06-2021,Amount_Avg_Transaction_Amount_01-07-2021_to_30-09-2021,Amount_Avg_Transaction_Amount_01-10-2021_to_31-12-2021,Amount_Avg_Transaction_Amount_01-01-2022_to_31-03-2022,Amount_Avg_Transaction_Amount_01-04-2022_to_30-06-2022,Amount_Avg_Transaction_Amount_01-07-2022_to_30-09-2022,Amount_Avg_Transaction_Amount_01-10-2022_to_31-12-2022,Category,Monthly_Income_Savings,Base_CID
0,1000001,14573.676250,36558.041890,39251.589470,49733.723200,34301.836400,67999.607570,72827.554870,64711.558480,0,...,2316.884595,1881.575106,1649.814369,1050.465067,1415.338750,1708.684791,1651.674990,Loyal Business,8538.204910,79434
1,1000002,12458.892350,29701.509290,32764.586630,49544.293270,23059.958250,33745.290180,40529.396050,45421.182750,0,...,1629.278333,1360.976211,1525.657970,1270.519612,1245.721327,1265.601867,1373.203747,Loyal Business,9212.229389,0
2,1000003,1715.065000,12380.019320,24418.109550,45223.428560,87343.521000,72131.871840,73478.580050,75201.235880,0,...,1073.085909,1269.972614,1212.456051,2268.190727,1740.961749,1860.892356,1850.673964,Loyal Business,9238.743117,0
3,1000004,967.139000,5177.223571,6395.929655,4744.074407,3617.540541,3696.674221,5579.091055,5599.246885,90,...,1161.095476,1534.231494,1254.579605,896.662973,905.488961,1310.355316,1454.196585,Loyal Business,8122.398130,33873
4,1000005,1978.103000,5769.430435,29163.155850,51935.832050,64781.767860,56975.955000,56152.353770,64502.964390,0,...,633.587391,1166.136064,2031.232923,1883.323932,1662.359962,1559.452592,1557.698898,Loyal Business,8992.601808,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10095,1112596,2957.237910,2770.348455,3837.783074,5899.426037,18630.836430,25921.268210,31305.401550,41808.971110,2,...,207.071273,224.598479,236.905958,453.098333,373.874464,395.895000,343.605882,Loyal Pension,3510.800000,25273
10096,1112597,2026.905726,2026.614130,2181.827164,3764.886860,9219.806829,13923.784580,16824.413770,23289.939770,3,...,197.651630,201.481828,208.627080,374.340976,280.031589,293.152963,252.224631,Loyal Pension,2834.150000,24853
10097,1112598,9399.204127,10141.575370,11238.514590,13083.411560,25542.850000,33624.614200,37357.300410,47189.179200,0,...,184.300071,184.751541,199.882594,499.451563,373.855800,398.466069,318.368400,Loyal Pension,2791.540000,28923
10098,1112599,5090.248289,5939.468870,7683.713775,10253.478420,25929.246410,36709.879920,40719.913900,50326.580550,0,...,166.609916,177.635591,194.606927,488.632051,318.350492,384.161585,322.573955,Loyal Pension,3343.300000,25951


In [22]:
df11 = merged_df[merged_df['Category'] == 'At-Risk Pension']
# Replace values in 'Column_X' from 1 till the count of rows
row_count = len(df11)  # Get the count of rows in the column
replacement_values = list(range(1, row_count + 1))  # Generate replacement values

df11['CID'] = replacement_values
df11['CID']

C:\Users\nitish.mallick\AppData\Local\Temp\ipykernel_17292\1754974303.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df11['CID'] = replacement_values


5000       1
5001       2
5002       3
5003       4
5004       5
        ... 
7495    2496
7496    2497
7497    2498
7498    2499
7499    2500
Name: CID, Length: 2500, dtype: int64

In [11]:
df3['CID']

0            1
1            2
2            3
3            4
4            5
         ...  
18745    18746
18746    18747
18747    18748
18748    18749
18749    18750
Name: CID, Length: 18750, dtype: int64

In [23]:
merged_df = pd.concat([df3,df4,df5,df6,df7,df8,df9,df10,df11], axis = 0)
merged_df

,CID,Balance,Balance_rolling_avg_01-04-2021_to_30-06-2021,Balance_rolling_avg_01-07-2021_to_30-09-2021,Balance_rolling_avg_01-10-2021_to_31-12-2021,Balance_rolling_avg_01-01-2022_to_31-03-2022,Balance_rolling_avg_01-04-2022_to_30-06-2022,Balance_rolling_avg_01-07-2022_to_30-09-2022,Balance_rolling_avg_01-10-2022_to_31-12-2022,Count_minimum_balance_Total,...,Amount_Avg_Transaction_Amount_01-04-2021_to_30-06-2021,Amount_Avg_Transaction_Amount_01-07-2021_to_30-09-2021,Amount_Avg_Transaction_Amount_01-10-2021_to_31-12-2021,Amount_Avg_Transaction_Amount_01-01-2022_to_31-03-2022,Amount_Avg_Transaction_Amount_01-04-2022_to_30-06-2022,Amount_Avg_Transaction_Amount_01-07-2022_to_30-09-2022,Amount_Avg_Transaction_Amount_01-10-2022_to_31-12-2022,Category,Monthly_Income_Savings,Base_CID
0,1,16308.722690,38916.864090,62852.127390,73946.328790,57369.608210,47937.740540,46194.914230,43240.838990,0,...,995.374242,1014.971338,866.734840,627.782663,494.358076,500.324133,530.027406,Loyal Salaried,2421.04,89979
1,2,9255.025667,16355.617110,18054.180580,22576.206370,28556.024460,21282.942290,19041.487810,17915.060680,0,...,430.872632,458.738013,349.039458,191.832114,232.768182,222.638603,231.608006,Loyal Salaried,3631.23,73809
2,3,28013.241940,55249.529890,75312.590660,124663.198000,241587.684500,243256.824900,271242.218800,308587.186200,0,...,1604.525217,1620.265882,1146.026904,639.595784,909.924877,850.575368,854.121479,Loyal Salaried,1992.66,93544
3,4,5717.198710,10528.762470,12948.687810,12278.112850,10580.049910,12246.359110,9335.939135,7390.403470,18,...,344.880941,364.535822,268.734164,200.734174,182.082620,168.461603,160.741735,Loyal Salaried,7147.78,43549
4,5,13120.715500,26183.725060,38748.892630,51155.582200,51726.342690,52290.028300,58225.006580,58595.673640,0,...,776.963563,917.990855,649.624167,570.415604,503.855167,415.706439,459.855270,Loyal Salaried,1165.74,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7495,2496,3160.441757,2193.945507,1876.915915,1608.797337,347.916545,292.717206,219.869437,166.547629,116,...,166.286232,159.353780,162.662714,71.737818,60.968382,34.345211,27.502474,At-Risk Pension,1547.05,28149
7496,2497,9199.987679,12816.235120,18186.098920,19450.639750,18670.473650,18757.018950,19482.204060,20163.352740,0,...,342.434876,285.647934,230.248182,96.766941,105.250484,116.524235,111.286587,At-Risk Pension,4359.45,75790
7497,2498,7146.261486,5511.867848,5727.027368,5716.065207,2868.321325,2192.102673,1803.531000,1279.336099,71,...,247.284873,240.382470,235.865533,86.873976,74.460891,64.821500,58.795248,At-Risk Pension,3103.70,29673
7498,2499,1903.310612,1484.730921,1178.952168,1041.275161,1427.585962,1658.351368,1639.398120,1407.205821,102,...,431.406184,356.041399,370.255936,380.502308,427.224421,445.922406,374.087214,At-Risk Pension,3424.20,76379


In [24]:
merged_df['CID'].max()

35000

In [25]:
merged_df.to_csv('Savings_Derived_with Base CID.csv', index = False)

In [40]:
merged_df = pd.read_csv('Savings_Derived_with Base CID.csv')

In [33]:
df2 = pd.read_csv('D:\All_Files\Base_Data\Base_Data_CIDs.csv')
df2

,CID,New_CID
0,1,CUST_ID1060
1,2,CUST_ID1721
2,3,CUST_ID1402
3,4,CUST_ID1063
4,5,CUST_ID1794
...,...,...
99995,77443,CUST_ID18399995
99996,77444,CUST_ID18899996
99997,77445,CUST_ID10299997
99998,77446,CUST_ID13299998


In [45]:
merged_df

,CID_x,Balance,Balance_rolling_avg_01-04-2021_to_30-06-2021,Balance_rolling_avg_01-07-2021_to_30-09-2021,Balance_rolling_avg_01-10-2021_to_31-12-2021,Balance_rolling_avg_01-01-2022_to_31-03-2022,Balance_rolling_avg_01-04-2022_to_30-06-2022,Balance_rolling_avg_01-07-2022_to_30-09-2022,Balance_rolling_avg_01-10-2022_to_31-12-2022,Count_minimum_balance_Total,...,Amount_Avg_Transaction_Amount_01-10-2021_to_31-12-2021,Amount_Avg_Transaction_Amount_01-01-2022_to_31-03-2022,Amount_Avg_Transaction_Amount_01-04-2022_to_30-06-2022,Amount_Avg_Transaction_Amount_01-07-2022_to_30-09-2022,Amount_Avg_Transaction_Amount_01-10-2022_to_31-12-2022,Category,Monthly_Income_Savings,Base_CID,CID_y,New_CID
0,1,16308.722690,38916.864090,62852.127390,73946.328790,57369.608210,47937.740540,46194.914230,43240.838990,0,...,866.734840,627.782663,494.358076,500.324133,530.027406,Loyal Salaried,2421.04,89979,89979.0,CUST_ID16559773
1,2,9255.025667,16355.617110,18054.180580,22576.206370,28556.024460,21282.942290,19041.487810,17915.060680,0,...,349.039458,191.832114,232.768182,222.638603,231.608006,Loyal Salaried,3631.23,73809,73809.0,CUST_ID14451054
2,3,28013.241940,55249.529890,75312.590660,124663.198000,241587.684500,243256.824900,271242.218800,308587.186200,0,...,1146.026904,639.595784,909.924877,850.575368,854.121479,Loyal Salaried,1992.66,93544,93544.0,CUST_ID19063338
3,4,5717.198710,10528.762470,12948.687810,12278.112850,10580.049910,12246.359110,9335.939135,7390.403470,18,...,268.734164,200.734174,182.082620,168.461603,160.741735,Loyal Salaried,7147.78,43549,43549.0,CUST_ID17620794
4,5,13120.715500,26183.725060,38748.892630,51155.582200,51726.342690,52290.028300,58225.006580,58595.673640,0,...,649.624167,570.415604,503.855167,415.706439,459.855270,Loyal Salaried,1165.74,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112595,2496,3160.441757,2193.945507,1876.915915,1608.797337,347.916545,292.717206,219.869437,166.547629,116,...,162.662714,71.737818,60.968382,34.345211,27.502474,At-Risk Pension,1547.05,28149,28149.0,CUST_ID10695566
112596,2497,9199.987679,12816.235120,18186.098920,19450.639750,18670.473650,18757.018950,19482.204060,20163.352740,0,...,230.248182,96.766941,105.250484,116.524235,111.286587,At-Risk Pension,4359.45,75790,75790.0,CUST_ID19098342
112597,2498,7146.261486,5511.867848,5727.027368,5716.065207,2868.321325,2192.102673,1803.531000,1279.336099,71,...,235.865533,86.873976,74.460891,64.821500,58.795248,At-Risk Pension,3103.70,29673,29673.0,CUST_ID11697090
112598,2499,1903.310612,1484.730921,1178.952168,1041.275161,1427.585962,1658.351368,1639.398120,1407.205821,102,...,370.255936,380.502308,427.224421,445.922406,374.087214,At-Risk Pension,3424.20,76379,76379.0,CUST_ID12998931


In [42]:
merged_df = merged_df.merge(df2, left_on='Base_CID', right_on = 'CID',how = 'left')

In [44]:
merged_df.fillna(0, inplace = True)

In [50]:
df1

,CID,Transaction Date,Transaction Type,Amount,Narration,Balance
0,1,2021-01-01,Credit,2021.98,BACS,5357.16
1,1,2021-01-01,Credit,267.86,Interest,5625.02
2,1,2021-01-02,Debit,31.50,Direct Debit_Autopay,5593.52
3,1,2021-01-02,Credit,89.25,Faster Payment,5682.77
4,1,2021-01-02,Debit,540.95,Standing Order_Rent,5141.82
...,...,...,...,...,...,...
2805765,5000,2023-01-27,Debit,150.71,Cash Withdrawal,39296.17
2805766,5000,2023-01-27,Debit,52.60,Fast Payment_Pharmacy/Healthcare,39243.57
2805767,5000,2023-01-28,Debit,45.26,Cash_Grocery,39198.31
2805768,5000,2023-01-29,Debit,18.18,Fast Payment_Travel,39180.13


In [74]:
df_from_dict = pd.DataFrame.from_dict(keys_values_dict, orient='index')
df_from_dict.reset_index()


,index,0
0,1,CUST_ID16559773
1,2,CUST_ID14451054
2,3,CUST_ID19063338
3,4,CUST_ID17620794
4,5,0
...,...,...
18745,18746,CUST_ID11549928
18746,18747,CUST_ID14841177
18747,18748,0
18748,18749,CUST_ID10718986


In [75]:
df_from_dict['index']

KeyError: 'index'

In [58]:
df4 = df4.merge(df_from_dict,left_on how = 'left' )

MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False

In [78]:
df

,CID_x,Balance,Balance_rolling_avg_01-04-2021_to_30-06-2021,Balance_rolling_avg_01-07-2021_to_30-09-2021,Balance_rolling_avg_01-10-2021_to_31-12-2021,Balance_rolling_avg_01-01-2022_to_31-03-2022,Balance_rolling_avg_01-04-2022_to_30-06-2022,Balance_rolling_avg_01-07-2022_to_30-09-2022,Balance_rolling_avg_01-10-2022_to_31-12-2022,Count_minimum_balance_Total,...,Amount_Avg_Transaction_Amount_01-10-2021_to_31-12-2021,Amount_Avg_Transaction_Amount_01-01-2022_to_31-03-2022,Amount_Avg_Transaction_Amount_01-04-2022_to_30-06-2022,Amount_Avg_Transaction_Amount_01-07-2022_to_30-09-2022,Amount_Avg_Transaction_Amount_01-10-2022_to_31-12-2022,Category,Monthly_Income_Savings,Base_CID,CID_y,New_CID
0,1,16308.722690,38916.86409,62852.12739,73946.32879,57369.60821,47937.74054,46194.914230,43240.83899,0,...,866.734840,627.782663,494.358076,500.324133,530.027406,Loyal Salaried,2421.04,89979,89979.0,CUST_ID16559773
1,2,9255.025667,16355.61711,18054.18058,22576.20637,28556.02446,21282.94229,19041.487810,17915.06068,0,...,349.039458,191.832114,232.768182,222.638603,231.608006,Loyal Salaried,3631.23,73809,73809.0,CUST_ID14451054
2,3,28013.241940,55249.52989,75312.59066,124663.19800,241587.68450,243256.82490,271242.218800,308587.18620,0,...,1146.026904,639.595784,909.924877,850.575368,854.121479,Loyal Salaried,1992.66,93544,93544.0,CUST_ID19063338
3,4,5717.198710,10528.76247,12948.68781,12278.11285,10580.04991,12246.35911,9335.939135,7390.40347,18,...,268.734164,200.734174,182.082620,168.461603,160.741735,Loyal Salaried,7147.78,43549,43549.0,CUST_ID17620794
4,5,13120.715500,26183.72506,38748.89263,51155.58220,51726.34269,52290.02830,58225.006580,58595.67364,0,...,649.624167,570.415604,503.855167,415.706439,459.855270,Loyal Salaried,1165.74,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18745,18746,29673.924530,44595.66948,60313.24962,62950.94054,83416.70500,127542.44630,150495.097500,157412.71450,0,...,920.242941,2172.592000,1084.769000,917.849450,1073.829387,Loyal Salaried,3776.69,72683,72683.0,CUST_ID11549928
18746,18747,9272.136606,12516.33632,24280.91702,33839.78427,85624.79722,112651.80940,106855.606200,106029.15880,0,...,499.552681,1277.551944,1040.863955,950.438224,1110.163791,Loyal Salaried,5765.14,63932,63932.0,CUST_ID14841177
18747,18748,30855.543630,46253.83799,65189.50063,71958.73808,95739.20563,124058.47690,115003.558100,116691.92380,0,...,867.325699,2071.844688,1548.519571,1363.681905,1454.587101,Loyal Salaried,1142.76,0,0.0,0
18748,18749,17349.531430,23346.28014,31062.71918,33534.81599,70775.15621,86015.93500,82875.751900,85142.25306,0,...,445.214708,1319.897241,865.530764,618.330408,701.268469,Loyal Salaried,6483.93,41741,41741.0,CUST_ID10718986


In [99]:
df = merged_df[merged_df['Category'] == 'At Risk Business']
df9 = pd.read_csv('D:\All_Files\Savings_Account\\2.0\Augment\At-Risk_Business.csv')
df9 = df9.merge(df[['CID_x','New_CID']], left_on = 'CID', right_on='CID_x', how = 'left' )
df9['Category'] = 'At-Risk Business'
# Replacing 'CID' column with 'New_CID' values
df9['CID'] = df9['New_CID']
# Drop 'New_CID' column if it's no longer needed
df9.drop(columns=['New_CID','CID_x'], inplace=True)
df9['CID']

0                        0
1                        0
2                        0
3                        0
4                        0
                ...       
1372477    CUST_ID11369950
1372478    CUST_ID11369950
1372479    CUST_ID11369950
1372480    CUST_ID11369950
1372481    CUST_ID11369950
Name: CID, Length: 1372482, dtype: object

In [100]:
df9.to_csv('At-Risk_Business.csv', index = False)

In [101]:
merged_df = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8,df9], axis = 0)
merged_df

,CID,Transaction Date,Transaction Type,Amount,Narration,Balance,Category
0,CUST_ID18795230,2021-01-01,Credit,2182.68,BACS,5240.27,At-Risk Pension
1,CUST_ID18795230,2021-01-01,Debit,30.25,Cash_Mobile Recharge,5210.02,At-Risk Pension
2,CUST_ID18795230,2021-01-01,Debit,22.32,Fast Payment_Grocery,5187.70,At-Risk Pension
3,CUST_ID18795230,2021-01-02,Credit,259.38,Interest,5447.08,At-Risk Pension
4,CUST_ID18795230,2021-01-03,Credit,49.29,Faster Payment,5496.37,At-Risk Pension
...,...,...,...,...,...,...,...
1372477,CUST_ID11369950,2022-08-01,Credit,0.00,Interest,0.01,At-Risk Business
1372478,CUST_ID11369950,2022-09-01,Credit,0.01,Interest,0.02,At-Risk Business
1372479,CUST_ID11369950,2022-10-01,Credit,0.01,Interest,0.03,At-Risk Business
1372480,CUST_ID11369950,2022-11-07,Credit,0.01,Interest,0.04,At-Risk Business


In [103]:
merged_df['CID'].nunique()

100001

In [102]:
merged_df.to_csv('Savings_Raw_with_Base_CID.csv', index = False)